# Preselection 

In [1]:
#from CMSJMECalculators import loadJMESystematicsCalculators
#loadJMESystematicsCalculators()

In [2]:
%%capture
!pip install rucio-clients

%%capture
#import ROOT
!pip3 install -U ipywidgets

In [3]:
from rucio.client.client import Client
import os
os.environ["RUCIO_HOME"] = "/cvmfs/cms.cern.ch/rucio/current/"
os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
os.environ['X509_USER_PROXY'] = "/opt/workspace/persistent-storage/UL/RDF-VBS_SSWW_lep_tauh/proxy"
RUCIO_SCOPE = "cms"
ACCOUNT = "ttedesch"

SITENAME = "T2_IT_Legnaro"

c = Client(account=ACCOUNT, auth_type="x509_proxy")

In [4]:
#!which root

In [5]:
#f = ROOT.TFile.Open("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/leptonSF/Muon_RunBCDEF_SF_ISO_2017.root")

In [6]:
#f.ls()

## Import libraries and choose whether to run locally or distributed

In [7]:
%%capture

import ROOT
import os
from dask.distributed import Client, PipInstall, WorkerPlugin
import json
from samplesUL import *
ROOT.RDF.Experimental.Distributed.open_files_locally = False
logger = ROOT.RDF.Experimental.Distributed.create_logger("DEBUG")
import sys 
sys.setrecursionlimit(1000000)


variationLimit = 120
distributed = False
MT = False
#redirector = "xrootd-cms.infn.it" 
redirector = "cmsrm-xrootd02.roma1.infn.it:7070"
maxNfilespersample = 9999
firstNfilesinchain= 1271
sched_port = 22980
nPartitions = 7*3
os.environ['X509_USER_PROXY'] = '/opt/workspace/persistent-storage/UL/RDF-VBS_SSWW_lep_tauh/proxy'
os.environ['X509_CERT_DIR'] = '/cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/'

In [8]:
#chain[:9]

In [9]:
if distributed != True and MT == True:
    ROOT.ROOT.EnableImplicitMT()

## Declare utils defined in a preselection.h file to the ROOT interpreter. If running distributed, connect to cluster and declare the file content in an initialization function

In [10]:
#from my_initialization_function_UL import *
text_file = open("preselection_UL.h", "r")
data = text_file.read()

text_file = open("preselection_part2_UL.h", "r")
data_2 = text_file.read()


def my_initialization_function():
    import ROOT
    import sys
    ROOT.EnableThreadSafety()
    ROOT.gInterpreter.AddIncludePath("/cvmfs/cms.dodas.infn.it/boost_1_77_0")
    sys.path.append("/lib/python3.8/site-packages")
    
    jec_prefix_MC = "Summer19UL17_V6_MC"
    jer_prefix_MC = "Summer19UL17_JRV3_MC"
    
    from urllib import request
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR2016aUL.txt", "RoccoR2016aUL.txt") 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR2016bUL.txt", "RoccoR2016bUL.txt") 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR2017UL.txt", "RoccoR2017UL.txt")
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR2018UL.txt", "RoccoR2018UL.txt") 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR.cc", "RoccoR.cc")
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR.h", "RoccoR.h")

    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/data/btagSF/DeepJet_106XUL16preVFPSF_v1_new.csv", "DeepJet_106XUL16preVFPSF_v1_new.csv")
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/data/btagSF/DeepJet_106XUL16postVFPSF_v2_new.csv", "DeepJet_106XUL16postVFPSF_v2_new.csv") 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/data/btagSF/DeepJet_106XUL17_v3_new.csv", "DeepJet_106XUL17_v3_new.csv") 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/nanoAOD-tools/data/btagSF/DeepJet_106XUL18_v2_new.csv", "DeepJet_106XUL18_v2_new.csv")
    
    
    ROOT.gInterpreter.Declare(
        '''
        #ifndef ROCCOR
        #define ROCCOR
        #include "RoccoR.cc"
        #endif
        ''')

    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC), "{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC))
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC), "{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_L3Absolute_AK4PFchs.txt".format(jec_prefix_MC), "{}_L3Absolute_AK4PFchs.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC), "{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC), "{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC))
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_SF_AK4PFchs.txt".format(jer_prefix_MC), "{}_SF_AK4PFchs.txt".format(jer_prefix_MC))
    
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_L1FastJet_AK8PFPuppi.txt".format(jec_prefix_MC), "{}_L1FastJet_AK8PFPuppi.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_L2Relative_AK8PFPuppi.txt".format(jec_prefix_MC), "{}_L2Relative_AK8PFPuppi.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_L3Absolute_AK8PFPuppi.txt".format(jec_prefix_MC), "{}_L3Absolute_AK8PFPuppi.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_UncertaintySources_AK8PFPuppi.txt".format(jec_prefix_MC), "{}_UncertaintySources_AK8PFPuppi.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_PtResolution_AK8PFPuppi.txt".format(jer_prefix_MC).format(jer_prefix_MC), "{}_PtResolution_AK8PFPuppi.txt".format(jer_prefix_MC))
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/jme_data/{}_SF_AK8PFPuppi.txt".format(jer_prefix_MC).format(jer_prefix_MC), "{}_SF_AK8PFPuppi.txt".format(jer_prefix_MC))
    
    
    ROOT.gInterpreter.Declare('{}'.format(data))
    

    
    ROOT.gInterpreter.ProcessLine('#ifndef LOADING')
    ROOT.gInterpreter.ProcessLine('#define LOADING')
    

    ROOT.gInterpreter.ProcessLine('reader_1_UL2017.load(calibration_UL2017, BTagEntry::FLAV_B, "comb");')
    ROOT.gInterpreter.ProcessLine('reader_1_UL2017.load(calibration_UL2017, BTagEntry::FLAV_C, "comb");')
    ROOT.gInterpreter.ProcessLine('reader_1_UL2017.load(calibration_UL2017, BTagEntry::FLAV_UDSG, "incl");')

    
    ROOT.gInterpreter.ProcessLine('#endif')
    
    ROOT.gInterpreter.Declare('{}'.format(data_2))
    
    #ROOT.gInterpreter.ProcessLine('#ifndef JME')
    #ROOT.gInterpreter.ProcessLine('#define JME')
    
    from CMSJMECalculators import loadJMESystematicsCalculators
    loadJMESystematicsCalculators()

    ROOT.gROOT.ProcessLine("JetVariationsCalculator myJetVarCalc{};")
    calc = getattr(ROOT, "myJetVarCalc")

    # redo JEC, push_back corrector parameters for different levels
    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L3Absolute_AK4PFchs.txt".format(jec_prefix_MC)))
    calc.setJEC(jecParams)

    uncert_sources = ["Total"]
    # calculate JES uncertainties
    with open("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)) as f:
        lines = f.read().split("\n")
        sources = [
            x for x in lines if x.startswith("[") and x.endswith("]")
        ]
        sources = [x[1:-1] for x in sources]
        sources = list(filter(lambda source: source in uncert_sources, sources))

    for s in sources:
        jcp_unc = ROOT.JetCorrectorParameters("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC), s)
        calc.addJESUncertainty(s, jcp_unc)

    # Smear jets, with JER uncertainty
    calc.setSmearing("./{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC), "./{}_SF_AK4PFchs.txt".format(jer_prefix_MC), False, True, 0.2, 3.)  # use hybrid recipe, matching parameters

    jetvariations = [calc.available()[i] for i in range(0,calc.available().size())]
    
    ROOT.gROOT.ProcessLine("Type1METVariationsCalculator myType1METVarCalc{};")
    calc = getattr(ROOT, "myType1METVarCalc")

    # redo JEC, push_back corrector parameters for different levels
    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L3Absolute_AK4PFchs.txt".format(jec_prefix_MC)))
    #jecParams.push_back(gbl.JetCorrectorParameters("./RDF-VBS_SSWW_lep_tauh/jme_data/{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)))
    calc.setJEC(jecParams)

    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC)))
    #jecParams.push_back(gbl.JetCorrectorParameters("./RDF-VBS_SSWW_lep_tauh/jme_data/{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)))
    calc.setL1JEC(jecParams)

    # calculate JES uncertainties
    with open("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)) as f:
        lines = f.read().split("\n")
        sources = [
            x for x in lines if x.startswith("[") and x.endswith("]")
        ]
        sources = [x[1:-1] for x in sources]
        sources = list(filter(lambda source: source in uncert_sources, sources))


    for s in sources:
        jcp_unc = ROOT.JetCorrectorParameters("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC), s)
        calc.addJESUncertainty(s, jcp_unc)

    # Smear jets, with JER uncertainty
    calc.setSmearing("./{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC), "./{}_SF_AK4PFchs.txt".format(jer_prefix_MC), False, True, 0.2, 3.)  # use hybrid recipe, matching parameters

    mett1variations = [calc.available()[i] for i in range(0, calc.available().size())]
    
    ROOT.gROOT.ProcessLine("Type1METVariationsCalculator myType1SmearMETVarCalc{};")
    calc = getattr(ROOT, "myType1SmearMETVarCalc")

    # redo JEC, push_back corrector parameters for different levels
    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L3Absolute_AK4PFchs.txt".format(jec_prefix_MC)))
    #jecParams.push_back(gbl.JetCorrectorParameters("./RDF-VBS_SSWW_lep_tauh/jme_data/{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)))
    calc.setJEC(jecParams)

    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC)))
    #jecParams.push_back(gbl.JetCorrectorParameters("./RDF-VBS_SSWW_lep_tauh/jme_data/{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)))
    calc.setL1JEC(jecParams)

    calc.setIsT1SmearedMET(True)

    # calculate JES uncertainties
    with open("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)) as f:
        lines = f.read().split("\n")
        sources = [
            x for x in lines if x.startswith("[") and x.endswith("]")
        ]
        sources = [x[1:-1] for x in sources]
        sources = list(filter(lambda source: source in uncert_sources, sources))


    for s in sources:
        jcp_unc = ROOT.JetCorrectorParameters("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC), s)
        calc.addJESUncertainty(s, jcp_unc)

    # Smear jets, with JER uncertainty
    calc.setSmearing("./{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC), "./{}_SF_AK4PFchs.txt".format(jer_prefix_MC), False, True, 0.2, 3.)  # use hybrid recipe, matching parameters

    mett1smearvariations = [calc.available()[i] for i in range(0, calc.available().size())]

    
    ROOT.gROOT.ProcessLine("FatJetVariationsCalculator myFatJetVarCalc{};")
    calc = getattr(ROOT, "myFatJetVarCalc")

    # redo JEC, push_back corrector parameters for different levels
    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK8PFPuppi.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L2Relative_AK8PFPuppi.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L3Absolute_AK8PFPuppi.txt".format(jec_prefix_MC)))
    #jecParams.push_back(gbl.JetCorrectorParameters("./RDF-VBS_SSWW_lep_tauh/jme_data/{}_UncertaintySources_AK4PFchs_AK4PFchs.txt".format(jec_prefix_MC)))
    calc.setJEC(jecParams)
    
    
    calc.setJMRValues(1.09, 1.14, 1.04);
    calc.setGMRValues(1.080, 1.161, 0.999)
    calc.setJMSValues(0.982, 0.986 , 0.978)
    calc.setGMSValues(0.983, 0.99, 0.976)
    
    calc.setAddHEM2018Issue(False);
    
    a = getattr(ROOT, "array<double,6>")()
    for i in range(6):
        a[i] = 0.
    reco_cen_params = getattr(ROOT, "array<double,6>")()
    reco_for_params = getattr(ROOT, "array<double,6>")()
    resol_cen_params = getattr(ROOT, "array<double,6>")()
    resol_for_params = getattr(ROOT, "array<double,6>")()
    for i,value in enumerate([1.0930197734452352, -0.00015006788774298745, 3.4486611503791434e-07, -2.681003093847412e-10, 8.674402290776817e-14, -1.0011358570698617e-17]):
        reco_cen_params[i] = value
    for i,value in enumerate([1.2721151537214315, -0.0005716403627542301, 8.372894123074334e-07, -5.204332049858346e-10, 1.4537520981877012e-13, -1.5038869243803616e-17]):
        reco_for_params[i] = value
    for i,value in enumerate([1.092735080341856, 4.142622682579229e-05, -1.3736805733597026e-07, 1.2295818250625584e-10, -4.197075395161288e-14, 4.923792745086088e-18]):
        resol_cen_params[i] = value
    for i,value in enumerate([1.1649278339374347, -0.00012678902807057208, 1.0594037344842974e-07, 6.072087019460118e-12, -1.992427482862693e-14, 3.644006510237158e-18]):
        resol_for_params[i] = value

    calc.setPuppiCorrections("1.0062610283313527-1.061605139842829*pow(x*0.07999000770091785,-1.2045376937033758)", reco_cen_params, reco_for_params, resol_cen_params, resol_for_params);

    # calculate JES uncertainties
    with open("./{}_UncertaintySources_AK8PFPuppi.txt".format(jec_prefix_MC)) as f:
        lines = f.read().split("\n")
        sources = [
            x for x in lines if x.startswith("[") and x.endswith("]")
        ]
        sources = [x[1:-1] for x in sources]
        sources = list(filter(lambda source: source in uncert_sources, sources))


    for s in sources:
        jcp_unc = ROOT.JetCorrectorParameters("./{}_UncertaintySources_AK8PFPuppi.txt".format(jec_prefix_MC), s)
        calc.addJESUncertainty(s, jcp_unc)

    # Smear jets, with JER uncertainty
    calc.setSmearing("./{}_PtResolution_AK8PFPuppi.txt".format(jer_prefix_MC), "./{}_SF_AK8PFPuppi.txt".format(jer_prefix_MC), False, True, 0.8, 3.)  # use hybrid recipe, matching parameters

    fatjetvariations = [calc.available()[i] for i in range(0,calc.available().size())]
    
    #ROOT.gInterpreter.ProcessLine('#endif')
    
    print(jetvariations)
    #return jetvariations, mett1variations, mett1smearvariations, fatjetvariations
    

jetvariations = [
 'nominal',
 'jerup',
 'jerdown',
 'jesTotalup',
 'jesTotaldown',
]

mett1variations = jetvariations 
mett1smearvariations = jetvariations
fatjetvariations = jetvariations

#jetvariations = jetvariations[:variationLimit]
#fatjetvariations = fatjetvariations[:variationLimit]
#mett1variations = mett1variations[:variationLimit]
#mett1smearvariations = mett1smearvariations[:variationLimit]

In [11]:
#client.restart()

In [12]:
%%time
# declare all the utils functions for the analysis to the ROOT interpreter. In the case of distributed execution this has to be done via ROOT.RDF.Experimental.Distributed.initialize function, which will initialize each worker properly.
from distributed.diagnostics.plugin import UploadFile

def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    print(working_dir)
    os.environ['X509_USER_PROXY'] = working_dir + '/proxy'
    os.system("export X509_USER_PROXY={}".format(working_dir + '/proxy'))
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    os.system("export X509_CERT_DIR=/cvmfs/grid.cern.ch/etc/grid-security/certificates/")
    #os.popen('cp {} {}'.format(working_dir + '/proxy', '../../../proxy'))
    shutil.copyfile(working_dir + '/proxy', working_dir + '/../../../proxy')
    return os.environ.get("X509_USER_PROXY")

def func():
    import sys
    sys.setrecursionlimit(1000000)

if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    client.register_worker_plugin(UploadFile("./proxy"))
    client.run(set_proxy)
    #import logging
    #from dask.distributed import LocalCluster
    #plugin_pickle = TestPlugin(1000000)
    #plugin_log = ErrorLogger(logging)
    #client.register_worker_plugin(plugin)
    #client.register_worker_plugin(plugin_pickle)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
    #jetvariations, mett1variations, mett1smearvariations, fatjetvariations = my_initialization_function()

    
else:
    RDataFrame = ROOT.RDataFrame
    #ROOT.gInterpreter.Declare('#include "postselection.h"')
    my_initialization_function()

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


['nominal', 'jerup', 'jerdown', 'jesTotalup', 'jesTotaldown']
CPU times: user 1min 2s, sys: 1.36 s, total: 1min 3s
Wall time: 1min 12s
Loading histogram NUM_TightID_DEN_TrackerMuons_abseta_pt from file https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/leptonSF/MuID_Tight_UL2017.root... 
Loading histogram NUM_IsoMu27_DEN_CutBasedIdTight_and_PFIsoTight_abseta_pt from file https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/leptonSF/MuTRIG_UL2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_abseta_pt from file https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/leptonSF/MuISO_Tight_UL2017.root... 
Loading histogram NUM_TrackerMuons_DEN_genTracks from file https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/leptonSF/MuRECO_UL2017.root... 
Loading histogram EGamma_SF2D from file https://vbs-pg-support.web.cern.ch/nanoAOD-tools/python/postprocessing/data/leptonSF/EleRECO_UL2017

In [14]:
#sources = filter(lambda source: source in uncert_sources, sources)

In [15]:
#client.register_worker_plugin(UploadFile("./preselection_function.py"))
#client.register_worker_plugin(UploadFile("./preselection_UL.h"))
#client.register_worker_plugin(UploadFile("./preselection_part2_UL.h"))

In [16]:
#sampleDict = {'WpWpJJ_EWK_2017': 0, 'WpWpJJ_QCD_2017': 1, 'VBS_SSWW_SM_2017': 2, 'VBS_SSWW_LL_SM_2017': 3, 'VBS_SSWW_TL_SM_2017': 4, 'VBS_SSWW_TT_SM_2017': 5, 'VBS_SSWW_cHW_2017': 6, 'VBS_SSWW_cHW_BSM_2017': 7, 'VBS_SSWW_cHW_INT_2017': 8, 'VBS_SSWW_cW_2017': 9, 'VBS_SSWW_cW_BSM_2017': 10, 'VBS_SSWW_cW_INT_2017': 11, 'VBS_SSWW_cW_cHW_2017': 12, 'VBS_SSWW_DIM6_2017': 13, 'VBS_SSWW_DIM6_SM_2017': 14, 'VBS_SSWW_cHW_SM_2017': 15, 'VBS_SSWW_cW_SM_2017': 16, 'VBS_SSWW_aQGC_2017': 17, 'VBS_SSWW_FS0_25_BSM_2017': 18, 'VBS_SSWW_FS0_25_INT_2017': 19, 'VBS_SSWW_FS0_25_SM_2017': 20, 'VBS_SSWW_FS0_25_2017': 21, 'VBS_SSWW_FS0_5_BSM_2017': 22, 'VBS_SSWW_FS0_5_INT_2017': 23, 'VBS_SSWW_FS0_5_SM_2017': 24, 'VBS_SSWW_FS0_5_2017': 25, 'VBS_SSWW_FS0_0_2017': 26, 'VBS_SSWW_FS1_50_BSM_2017': 27, 'VBS_SSWW_FS1_50_INT_2017': 28, 'VBS_SSWW_FS1_50_SM_2017': 29, 'VBS_SSWW_FS1_50_2017': 30, 'VBS_SSWW_FS1_10_BSM_2017': 31, 'VBS_SSWW_FS1_10_INT_2017': 32, 'VBS_SSWW_FS1_10_SM_2017': 33, 'VBS_SSWW_FS1_10_2017': 34, 'VBS_SSWW_FS1_0_2017': 35, 'VBS_SSWW_FM0_25_BSM_2017': 36, 'VBS_SSWW_FM0_25_INT_2017': 37, 'VBS_SSWW_FM0_25_SM_2017': 38, 'VBS_SSWW_FM0_25_2017': 39, 'VBS_SSWW_FM0_5_BSM_2017': 40, 'VBS_SSWW_FM0_5_INT_2017': 41, 'VBS_SSWW_FM0_5_SM_2017': 42, 'VBS_SSWW_FM0_5_2017': 43, 'VBS_SSWW_FM0_0_2017': 44, 'VBS_SSWW_FM1_25_BSM_2017': 45, 'VBS_SSWW_FM1_25_INT_2017': 46, 'VBS_SSWW_FM1_25_SM_2017': 47, 'VBS_SSWW_FM1_25_2017': 48, 'VBS_SSWW_FM1_5_BSM_2017': 49, 'VBS_SSWW_FM1_5_INT_2017': 50, 'VBS_SSWW_FM1_5_SM_2017': 51, 'VBS_SSWW_FM1_5_2017': 52, 'VBS_SSWW_FM1_0_2017': 53, 'VBS_SSWW_FM6_25_BSM_2017': 54, 'VBS_SSWW_FM6_25_INT_2017': 55, 'VBS_SSWW_FM6_25_SM_2017': 56, 'VBS_SSWW_FM6_25_2017': 57, 'VBS_SSWW_FM6_5_BSM_2017': 58, 'VBS_SSWW_FM6_5_INT_2017': 59, 'VBS_SSWW_FM6_5_SM_2017': 60, 'VBS_SSWW_FM6_5_2017': 61, 'VBS_SSWW_FM6_0_2017': 62, 'VBS_SSWW_FM7_50_BSM_2017': 63, 'VBS_SSWW_FM7_50_INT_2017': 64, 'VBS_SSWW_FM7_50_SM_2017': 65, 'VBS_SSWW_FM7_50_2017': 66, 'VBS_SSWW_FM7_10_BSM_2017': 67, 'VBS_SSWW_FM7_10_INT_2017': 68, 'VBS_SSWW_FM7_10_SM_2017': 69, 'VBS_SSWW_FM7_10_2017': 70, 'VBS_SSWW_FM7_0_2017': 71, 'VBS_SSWW_FT0_2p5_BSM_2017': 72, 'VBS_SSWW_FT0_2p5_INT_2017': 73, 'VBS_SSWW_FT0_2p5_SM_2017': 74, 'VBS_SSWW_FT0_2p5_2017': 75, 'VBS_SSWW_FT0_0p5_BSM_2017': 76, 'VBS_SSWW_FT0_0p5_INT_2017': 77, 'VBS_SSWW_FT0_0p5_SM_2017': 78, 'VBS_SSWW_FT0_0p5_2017': 79, 'VBS_SSWW_FT0_0_2017': 80, 'VBS_SSWW_FT1_1_BSM_2017': 81, 'VBS_SSWW_FT1_1_INT_2017': 82, 'VBS_SSWW_FT1_1_SM_2017': 83, 'VBS_SSWW_FT1_1_2017': 84, 'VBS_SSWW_FT1_0p2_BSM_2017': 85, 'VBS_SSWW_FT1_0p2_INT_2017': 86, 'VBS_SSWW_FT1_0p2_SM_2017': 87, 'VBS_SSWW_FT1_0p2_2017': 88, 'VBS_SSWW_FT1_0_2017': 89, 'VBS_SSWW_FT2_2p5_BSM_2017': 90, 'VBS_SSWW_FT2_2p5_INT_2017': 91, 'VBS_SSWW_FT2_2p5_SM_2017': 92, 'VBS_SSWW_FT2_2p5_2017': 93, 'VBS_SSWW_FT2_0p5_BSM_2017': 94, 'VBS_SSWW_FT2_0p5_INT_2017': 95, 'VBS_SSWW_FT2_0p5_SM_2017': 96, 'VBS_SSWW_FT2_0p5_2017': 97, 'VBS_SSWW_FT2_0_2017': 98, 'DHiggsToWW_2017': 99, 'sm_2017': 100, 'sm_lin_quad_cW_2017': 101, 'quad_cW_2017': 102, 'sm_lin_quad_cHW_2017': 103, 'quad_cHW_2017': 104, 'QCD_2017': 105, 'QCDHT_100to200_2017': 106, 'QCDHT_200to300_2017': 107, 'QCDHT_300to500_2017': 108, 'QCDHT_500to700_2017': 109, 'QCDHT_700to1000_2017': 110, 'QCDHT_1000to1500_2017': 111, 'QCDHT_1500to2000_2017': 112, 'QCDHT_2000toInf_2017': 113, 'ZZtoLep_2017': 114, 'ZZTo2L2Nu_2017': 115, 'ZZJJTo4L_EWK_2017': 116, 'ZZJJTo4L_QCD_2017': 117, 'GluGluToContinToZZTo2e2nu_2017': 118, 'GluGluToContinToZZTo2e2mu_2017': 119, 'GluGluToContinToZZTo2e2tau_2017': 120, 'GluGluToContinToZZTo2mu2nu_2017': 121, 'GluGluToContinToZZTo2mu2tau_2017': 122, 'GluGluToContinToZZTo4e_2017': 123, 'GluGluToContinToZZTo4mu_2017': 124, 'GluGluToContinToZZTo4tau_2017': 125, 'GluGluToContinToZZTo4L_2017': 126, 'TT_2017': 127, 'TT_SemiLep2017': 128, 'TT_Had_2017': 129, 'TTTo2L2Nu_2017': 130, 'TT_beff_2017': 131, 'WJets_2017': 132, 'WJetsHT70to100_2017': 133, 'WJetsHT100to200_2017': 134, 'WJetsHT200to400_2017': 135, 'WJetsHT400to600_2017': 136, 'WJetsHT600to800_2017': 137, 'WJetsHT800to1200_2017': 138, 'WJetsHT1200to2500_2017': 139, 'WJetsHT2500toInf_2017': 140, 'WJets_Fake_2017': 141, 'WJetsHT70to100_Fake_2017': 142, 'WJetsHT100to200_Fake_2017': 143, 'WJetsHT200to400_Fake_2017': 144, 'WJetsHT400to600_Fake_2017': 145, 'WJetsHT600to800_Fake_2017': 146, 'WJetsHT800to1200_Fake_2017': 147, 'WJetsHT1200to2500_Fake_2017': 148, 'WJetsHT2500toInf_Fake_2017': 149, 'DYJetsToLL_2017': 150, 'DY1JetsToLL_2017': 151, 'DY2JetsToLL_2017': 152, 'DY3JetsToLL_2017': 153, 'DY4JetsToLL_2017': 154, 'DYJetsToLLM5to50_2017': 155, 'DYJetsToLL_Fake_2017': 156, 'DY1JetsToLL_Fake_2017': 157, 'DY2JetsToLL_Fake_2017': 158, 'DY3JetsToLL_Fake_2017': 159, 'DY4JetsToLL_Fake_2017': 160, 'VG_2017': 161, 'ZG_2017': 162, 'WG_2017': 163, 'TVX_2017': 164, 'TTGJets_2017': 165, 'TTZToQQ_2017': 166, 'TTZToLLNuNu_2017': 167, 'TTWJetsToQQ_2017': 168, 'TTWJetsToLNu_2017': 169, 'tZq_ll_4f_2017': 170, 'WrongSign_2017': 171, 'WWto2L2Nu_2017': 172, 'GluGluToWWToENuENu_2017': 173, 'GluGluToWWToENuMNu2017': 174, 'GluGluToWWToENuTNu2017': 175, 'GluGluToWWToMNuENu_2017': 176, 'GluGluToWWToMNuMNu2017': 177, 'GluGluToWWToMNuTNu2017': 178, 'GluGluToWWToTNuENu_2017': 179, 'GluGluToWWToTNuMNu2017': 180, 'GluGluToWWToTNuTNu2017': 181, 'STtW_top_2017': 182, 'GluGluHToWWTo2L2Nu_2017': 183, 'GluGluHToZZTo2L2Q_2017': 184, 'GluGluHToZZTo4L_2017': 185, 'GluGluHToTauTau_2017': 186, 'VBFHToWWTo2L2Nu_2017': 187, 'VBFHToTauTau_2017': 188, 'ttHToNonbb_2017': 189, 'VHToNonbb_2017': 190, 'OtherWS_2017': 191, 'Other_2017': 192, 'WWTo2L2Nu_DoubleScattering_2017': 193, 'WWW_4F_2017': 194, 'WWZTo3L1Nu2Q_2017': 195, 'WZZ_2017': 196, 'ZZZ_2017': 197, 'WWG_2017': 198, 'WZ_2017': 199, 'FakeMu_2017': 200, 'DataMu_2017': 201, 'DataMuB_2017': 202, 'DataMuC_2017': 203, 'DataMuD_2017': 204, 'DataMuE_2017': 205, 'DataMuF_2017': 206, 'DataMuFake_2017': 207, 'FakeEle_2017': 208, 'FakeMuPromptTau_2017': 209, 'PromptMuFakeTau_2017': 210, 'FakeMuFakeTau_2017': 211, 'DataEle_2017': 212, 'DataEleB_2017': 213, 'DataEleC_2017': 214, 'DataEleD_2017': 215, 'DataEleE_2017': 216, 'DataEleF_2017': 217, 'DataEleFake_2017': 218, 'MCFake_2017': 219, 'FakeElePromptTau_2017': 220, 'PromptEleFakeTau_2017': 221, 'FakeEleFakeTau_2017': 222, 'DataEleMu_2017': 223, 'FakeEleMu_2017': 224, 'DataTau_2017': 225, 'DataTauB_2017': 226, 'DataTauC_2017': 227, 'DataTauD_2017': 228, 'DataTauE_2017': 229, 'DataTauF_2017': 230, 'DataTauFake_2017': 231, 'DataHT_2017': 232, 'DataHTnoB_2017': 233, 'DataHTB_2017': 234, 'DataHTC_2017': 235, 'DataHTD_2017': 236, 'DataHTE_2017': 237, 'DataHTF_2017': 238, 'DataMET_2017': 239, 'DataMETB_2017': 240, 'DataMETC_2017': 241, 'DataMETD_2017': 242, 'DataMETE_2017': 243, 'DataMETF_2017': 244, 'SampleEleFake_2017': 245, 'SampleMuFake_2017': 246, 'SampleHTFake_2017': 247, 'SampleHTFakepart_2017': 248, 'WpWpJJ_EWK_2018': 249, 'WpWpJJ_QCD_2018': 250, 'VBS_SSWW_SM_2018': 251, 'VBS_SSWW_LL_SM_2018': 252, 'VBS_SSWW_TL_SM_2018': 253, 'VBS_SSWW_TT_SM_2018': 254, 'VBS_SSWW_cHW_2018': 255, 'VBS_SSWW_cHW_BSM_2018': 256, 'VBS_SSWW_cHW_INT_2018': 257, 'VBS_SSWW_cW_2018': 258, 'VBS_SSWW_cW_BSM_2018': 259, 'VBS_SSWW_cW_INT_2018': 260, 'VBS_SSWW_cW_cHW_2018': 261, 'VBS_SSWW_DIM6_2018': 262, 'VBS_SSWW_DIM6_SM_2018': 263, 'VBS_SSWW_cHW_SM_2018': 264, 'VBS_SSWW_cW_SM_2018': 265, 'VBS_SSWW_aQGC_2018': 266, 'VBS_SSWW_FS0_25_BSM_2018': 267, 'VBS_SSWW_FS0_25_INT_2018': 268, 'VBS_SSWW_FS0_25_SM_2018': 269, 'VBS_SSWW_FS0_25_2018': 270, 'VBS_SSWW_FS0_5_BSM_2018': 271, 'VBS_SSWW_FS0_5_INT_2018': 272, 'VBS_SSWW_FS0_5_SM_2018': 273, 'VBS_SSWW_FS0_5_2018': 274, 'VBS_SSWW_FS0_0_2018': 275, 'VBS_SSWW_FS1_50_BSM_2018': 276, 'VBS_SSWW_FS1_50_INT_2018': 277, 'VBS_SSWW_FS1_50_SM_2018': 278, 'VBS_SSWW_FS1_50_2018': 279, 'VBS_SSWW_FS1_10_BSM_2018': 280, 'VBS_SSWW_FS1_10_INT_2018': 281, 'VBS_SSWW_FS1_10_SM_2018': 282, 'VBS_SSWW_FS1_10_2018': 283, 'VBS_SSWW_FS1_0_2018': 284, 'VBS_SSWW_FM0_25_BSM_2018': 285, 'VBS_SSWW_FM0_25_INT_2018': 286, 'VBS_SSWW_FM0_25_SM_2018': 287, 'VBS_SSWW_FM0_25_2018': 288, 'VBS_SSWW_FM0_5_BSM_2018': 289, 'VBS_SSWW_FM0_5_INT_2018': 290, 'VBS_SSWW_FM0_5_SM_2018': 291, 'VBS_SSWW_FM0_5_2018': 292, 'VBS_SSWW_FM0_0_2018': 293, 'VBS_SSWW_FM1_25_BSM_2018': 294, 'VBS_SSWW_FM1_25_INT_2018': 295, 'VBS_SSWW_FM1_25_SM_2018': 296, 'VBS_SSWW_FM1_25_2018': 297, 'VBS_SSWW_FM1_5_BSM_2018': 298, 'VBS_SSWW_FM1_5_INT_2018': 299, 'VBS_SSWW_FM1_5_SM_2018': 300, 'VBS_SSWW_FM1_5_2018': 301, 'VBS_SSWW_FM1_0_2018': 302, 'VBS_SSWW_FM6_25_BSM_2018': 303, 'VBS_SSWW_FM6_25_INT_2018': 304, 'VBS_SSWW_FM6_25_SM_2018': 305, 'VBS_SSWW_FM6_25_2018': 306, 'VBS_SSWW_FM6_5_BSM_2018': 307, 'VBS_SSWW_FM6_5_INT_2018': 308, 'VBS_SSWW_FM6_5_SM_2018': 309, 'VBS_SSWW_FM6_5_2018': 310, 'VBS_SSWW_FM6_0_2018': 311, 'VBS_SSWW_FM7_50_BSM_2018': 312, 'VBS_SSWW_FM7_50_INT_2018': 313, 'VBS_SSWW_FM7_50_SM_2018': 314, 'VBS_SSWW_FM7_50_2018': 315, 'VBS_SSWW_FM7_10_BSM_2018': 316, 'VBS_SSWW_FM7_10_INT_2018': 317, 'VBS_SSWW_FM7_10_SM_2018': 318, 'VBS_SSWW_FM7_10_2018': 319, 'VBS_SSWW_FM7_0_2018': 320, 'VBS_SSWW_FT0_2p5_BSM_2018': 321, 'VBS_SSWW_FT0_2p5_INT_2018': 322, 'VBS_SSWW_FT0_2p5_SM_2018': 323, 'VBS_SSWW_FT0_2p5_2018': 324, 'VBS_SSWW_FT0_0p5_BSM_2018': 325, 'VBS_SSWW_FT0_0p5_INT_2018': 326, 'VBS_SSWW_FT0_0p5_SM_2018': 327, 'VBS_SSWW_FT0_0p5_2018': 328, 'VBS_SSWW_FT0_0_2018': 329, 'VBS_SSWW_FT1_1_BSM_2018': 330, 'VBS_SSWW_FT1_1_INT_2018': 331, 'VBS_SSWW_FT1_1_SM_2018': 332, 'VBS_SSWW_FT1_1_2018': 333, 'VBS_SSWW_FT1_0p2_BSM_2018': 334, 'VBS_SSWW_FT1_0p2_INT_2018': 335, 'VBS_SSWW_FT1_0p2_SM_2018': 336, 'VBS_SSWW_FT1_0p2_2018': 337, 'VBS_SSWW_FT1_0_2018': 338, 'VBS_SSWW_FT2_2p5_BSM_2018': 339, 'VBS_SSWW_FT2_2p5_INT_2018': 340, 'VBS_SSWW_FT2_2p5_SM_2018': 341, 'VBS_SSWW_FT2_2p5_2018': 342, 'VBS_SSWW_FT2_0p5_BSM_2018': 343, 'VBS_SSWW_FT2_0p5_INT_2018': 344, 'VBS_SSWW_FT2_0p5_SM_2018': 345, 'VBS_SSWW_FT2_0p5_2018': 346, 'VBS_SSWW_FT2_0_2018': 347, 'DHiggsToWW_2018': 348, 'sm_2018': 349, 'sm_lin_quad_cW_2018': 350, 'quad_cW_2018': 351, 'sm_lin_quad_cHW_2018': 352, 'quad_cHW_2018': 353, 'QCD_2018': 354, 'QCDHT_100to200_2018': 355, 'QCDHT_200to300_2018': 356, 'QCDHT_300to500_2018': 357, 'QCDHT_500to700_2018': 358, 'QCDHT_700to1000_2018': 359, 'QCDHT_1000to1500_2018': 360, 'QCDHT_1500to2000_2018': 361, 'QCDHT_2000toInf_2018': 362, 'ZZtoLep_2018': 363, 'ZZTo2L2Nu_2018': 364, 'ZZJJTo4L_EWK_2018': 365, 'ZZJJTo4L_QCD_2018': 366, 'GluGluToContinToZZTo2e2nu_2018': 367, 'GluGluToContinToZZTo2e2mu_2018': 368, 'GluGluToContinToZZTo2e2tau_2018': 369, 'GluGluToContinToZZTo2mu2nu_2018': 370, 'GluGluToContinToZZTo2mu2tau_2018': 371, 'GluGluToContinToZZTo4e_2018': 372, 'GluGluToContinToZZTo4mu_2018': 373, 'GluGluToContinToZZTo4tau_2018': 374, 'GluGluToContinToZZTo4L_2018': 375, 'TT_2018': 376, 'TT_SemiLep_2018': 377, 'TT_Had_2018': 378, 'WJets_2018': 379, 'WJetsHT70to100_2018': 380, 'WJetsHT100to200_2018': 381, 'WJetsHT200to400_2018': 382, 'WJetsHT400to600_2018': 383, 'WJetsHT600to800_2018': 384, 'WJetsHT800to1200_2018': 385, 'WJetsHT1200to2500_2018': 386, 'WJetsHT2500toInf_2018': 387, 'DYJetsToLL_2018': 388, 'DY1JetsToLL_2018': 389, 'DY2JetsToLL_2018': 390, 'DY3JetsToLL_2018': 391, 'DY4JetsToLL_2018': 392, 'SampleHTFake_2018': 393, 'VG_2018': 394, 'ZG_2018': 395, 'WG_2018': 396, 'TVX_2018': 397, 'TTGJets_2018': 398, 'TTZToQQ_2018': 399, 'TTZToLLNuNu_2018': 400, 'TTWJetsToQQ_2018': 401, 'TTWJetsToLNu_2018': 402, 'tZq_ll_4f_2018': 403, 'WrongSign_2018': 404, 'WWto2L2Nu_2018': 405, 'GluGluToWWToENuENu_2018': 406, 'GluGluToWWToMNuENu_2018': 407, 'GluGluToWWToTNuENu_2018': 408, 'STtW_top_2018': 409, 'GluGluHToWWTo2L2Nu_2018': 410, 'GluGluHToZZTo2L2Q_2018': 411, 'GluGluHToZZTo4L_2018': 412, 'GluGluHToTauTau_2018': 413, 'VBFHToWWTo2L2Nu_2018': 414, 'VBFHToTauTau_2018': 415, 'ttHToNonbb_2018': 416, 'VHToNonbb_2018': 417, 'OtherWS_2018': 418, 'Other_2018': 419, 'WWTo2L2Nu_DoubleScattering_2018': 420, 'WWW_4F_2018': 421, 'WWZTo3L1Nu2Q_2018': 422, 'WZZ_2018': 423, 'ZZZ_2018': 424, 'WWG_2018': 425, 'TTTo2L2Nu_2018': 426, 'WZ_2018': 427, 'FakeMu_2018': 428, 'DataMu_2018': 429, 'DataMuA_2018': 430, 'DataMuB_2018': 431, 'DataMuC_2018': 432, 'DataMuD_2018': 433, 'FakeEle_2018': 434, 'FakeMuPromptTau_2018': 435, 'PromptMuFakeTau_2018': 436, 'FakeMuFakeTau_2018': 437, 'DataEle_2018': 438, 'DataEleA_2018': 439, 'DataEleB_2018': 440, 'DataEleC_2018': 441, 'DataEleD_2018': 442, 'FakeElePromptTau_2018': 443, 'PromptEleFakeTau_2018': 444, 'FakeEleFakeTau_2018': 445, 'DataHT_2018': 446, 'DataHTA_2018': 447, 'DataHTB_2018': 448, 'DataHTC_2018': 449, 'DataHTD_2018': 450}

## Define samples to be processed

In [17]:
sampleDict = {'ZZtoLep_UL2016APV': 0,'ZZTo2L2Nu_UL2016APV': 1,'ZZTo4L_UL2016APV': 2,'GluGluToContinToZZTo4e_UL2016APV': 3,'GluGluToContinToZZTo2e2mu_UL2016APV': 4,'GluGluToContinToZZTo2e2tau_UL2016APV': 5,'GluGluToContinToZZTo2mu2nu_UL2016APV': 6,'GluGluToContinToZZTo4mu_UL2016APV': 7,'GluGluToContinToZZTo2mu2tau_UL2016APV': 8,'GluGluToContinToZZTo2tau2nu_UL2016APV': 9,'GluGluToContinToZZTo4tau_UL2016APV': 10,'GluGluToContinToZZTo2e2nu_UL2016APV': 11,'TT_UL2016APV': 12,'TT_SemiLep_UL2016APV': 13,'TT_Had_UL2016APV': 14,'TTTo2L2Nu_UL2016APV': 15,'TT_beff_UL2016APV': 16,'TVX_UL2016APV': 17,'TTGJets_UL2016APV': 18,'TTZToQQ_UL2016APV': 19,'TTZToLLNuNu_UL2016APV': 20,'TTWJetsToQQ_UL2016APV': 21,'TTWJetsToLNu_UL2016APV': 22,'tZq_ll_4f_UL2016APV': 23,'VG_UL2016APV': 24,'ZG_UL2016APV': 25,'WG_UL2016APV': 26,'WrongSign_UL2016APV': 27,'WWto2L2Nu_UL2016APV': 28,'GluGluToWWToENEN_UL2016APV': 29,'GluGluToWWToENMN_UL2016APV': 30,'GluGluToWWToENTN_UL2016APV': 31,'GluGluToWWToMNEN_UL2016APV': 32,'GluGluToWWToMNMN_UL2016APV': 33,'GluGluToWWToMNTN_UL2016APV': 34,'GluGluToWWToTNEN_UL2016APV': 35,'GluGluToWWToTNMN_UL2016APV': 36,'GluGluToWWToTNTN_UL2016APV': 37,'ST_tW_top_UL2016APV': 38,'ST_tW_antitop_UL2016APV': 39,'GluGluHToWWTo2L2Nu_UL2016APV': 40,'GluGluHToWWToLNuQQ_UL2016APV': 41,'GluGluHToZZTo4L_UL2016APV': 42,'GluGluHToTauTau_UL2016APV': 43,'VBFHToWWTo2L2Nu_UL2016APV': 44,'VBFHToTauTau_UL2016APV': 45,'ttHToNonbb_UL2016APV': 46,'VHToNonbb_UL2016APV': 47,'Triboson_UL2016APV': 48,'WWTo2L2Nu_DoubleScattering_UL2016': 49,'WWW_4F_UL2016APV': 50,'WWZ_4F_UL2016APV': 51,'WZZ_UL2016APV': 52,'ZZZ_UL2016APV': 53,'WWG_UL2016APV': 54,'WJets_UL2016APV': 55,'WJetsHT70to100_UL2016APV': 56,'WJetsHT100to200_UL2016APV': 57,'WJetsHT200to400_UL2016APV': 58,'WJetsHT400to600_UL2016APV': 59,'WJetsHT600to800_UL2016APV': 60,'WJetsHT800to1200_UL2016APV': 61,'WJetsHT1200to2500_UL2016APV': 62,'WJetsHT2500toInf_UL2016APV': 63,'WZ_UL2016APV': 64,'DYJetsToLL_UL2016APV': 65,'DYJetsToLL_FxFx_UL2016APV': 65,'DYJetsToLL_M10to50_UL2016APV': 66,'DYJetsToLL_M50_UL2016APV': 67,'DYJetsToLL_M50_FxFx_UL2016APV': 67,'DYJetsToLL_M50_UL2016APV_ext': 67,'WpWpJJ_EWK_UL2016APV': 68,'WpWpJJ_QCD_UL2016APV': 69,'VBS_SSWW_SM_UL2016APV': 70,'VBS_SSWW_LL_SM_UL2016APV': 71,'VBS_SSWW_TL_SM_UL2016APV': 72,'VBS_SSWW_TT_SM_UL2016APV': 73,'VBS_SSWW_cW_UL2016APV': 74,'VBS_SSWW_cW_SM_UL2016APV': 75,'VBS_SSWW_cW_BSM_UL2016APV': 76,'VBS_SSWW_cW_INT_UL2016APV': 77,'VBS_SSWW_cHW_UL2016APV': 78,'VBS_SSWW_cHW_SM_UL2016APV': 79,'VBS_SSWW_cHW_BSM_UL2016APV': 80,'VBS_SSWW_cHW_INT_UL2016APV': 81,'VBS_SSWW_cW_cHW_UL2016APV': 82,'VBS_SSWW_DIM6_UL2016APV': 83,'VBS_SSWW_DIM6_SM_UL2016APV': 84,'ZZtoLep_UL2016': 85,'ZZTo2L2Nu_UL2016': 86,'ZZTo4L_UL2016': 87,'GluGluToContinToZZTo4e_UL2016': 88,'GluGluToContinToZZTo2e2mu_UL2016': 89,'GluGluToContinToZZTo2e2tau_UL2016': 90,'GluGluToContinToZZTo2mu2nu_UL2016': 91,'GluGluToContinToZZTo4mu_UL2016': 92,'GluGluToContinToZZTo2mu2tau_UL2016': 93,'GluGluToContinToZZTo2tau2nu_UL2016': 94,'GluGluToContinToZZTo4tau_UL2016': 95,'GluGluToContinToZZTo2e2nu_UL2016': 96,'TT_UL2016': 97,'TT_SemiLep_UL2016': 98,'TT_Had_UL2016': 99,'TTTo2L2Nu_UL2016': 100,'TT_beff_UL2016': 101,'TVX_UL2016': 102,'TTGJets_UL2016': 103,'TTZToQQ_UL2016': 104,'TTZToLLNuNu_UL2016': 105,'TTWJetsToQQ_UL2016': 106,'TTWJetsToLNu_UL2016': 107,'tZq_ll_4f_UL2016': 108,'VG_UL2016': 109,'ZG_UL2016': 110,'WG_UL2016': 111,'WrongSign_UL2016': 112,'WWto2L2Nu_UL2016': 113,'GluGluToWWToENEN_UL2016': 114,'GluGluToWWToENMN_UL2016': 115,'GluGluToWWToENTN_UL2016': 116,'GluGluToWWToMNEN_UL2016': 117,'GluGluToWWToMNMN_UL2016': 118,'GluGluToWWToMNTN_UL2016': 119,'GluGluToWWToTNEN_UL2016': 120,'GluGluToWWToTNMN_UL2016': 121,'GluGluToWWToTNTN_UL2016': 122,'ST_tW_top_UL2016': 123,'ST_tW_antitop_UL2016': 124,'GluGluHToWWTo2L2Nu_UL2016': 125,'GluGluHToWWToLNuQQ_UL2016': 126,'GluGluHToZZTo4L_UL2016': 127,'GluGluHToTauTau_UL2016': 128,'VBFHToWWTo2L2Nu_UL2016': 129,'VBFHToTauTau_UL2016': 130,'ttHToNonbb_UL2016': 131,'VHToNonbb_UL2016': 132,'Triboson_UL2016': 133,'WWW_4F_UL2016': 134,'WWZ_4F_UL2016': 135,'WZZ_UL2016': 136,'ZZZ_UL2016': 137,'WWG_UL2016': 138,'WJets_UL2016': 139,'WJetsHT70to100_UL2016': 140,'WJetsHT100to200_UL2016': 141,'WJetsHT200to400_UL2016': 142,'WJetsHT400to600_UL2016': 143,'WJetsHT600to800_UL2016': 144,'WJetsHT800to1200_UL2016': 145,'WJetsHT1200to2500_UL2016': 146,'WJetsHT2500toInf_UL2016': 147,'WZ_UL2016': 148,'DYJetsToLL_UL2016': 149,'DYJetsToLL_FxFx_UL2016': 149,'DYJetsToLL_M10to50_UL2016': 150,'DYJetsToLL_M50_UL2016': 151,'DYJetsToLL_M50_FxFx_UL2016': 151,'DYJetsToLL_M50_UL2016_ext': 151,'WpWpJJ_EWK_UL2016': 152,'WpWpJJ_QCD_UL2016': 153,'VBS_SSWW_SM_UL2016': 154,'VBS_SSWW_LL_SM_UL2016': 155,'VBS_SSWW_TL_SM_UL2016': 156,'VBS_SSWW_TT_SM_UL2016': 157,'VBS_SSWW_cW_UL2016': 158,'VBS_SSWW_cW_SM_UL2016': 159,'VBS_SSWW_cW_BSM_UL2016': 160,'VBS_SSWW_cW_INT_UL2016': 161,'VBS_SSWW_cHW_UL2016': 162,'VBS_SSWW_cHW_SM_UL2016': 163,'VBS_SSWW_cHW_BSM_UL2016': 164,'VBS_SSWW_cHW_INT_UL2016': 165,'VBS_SSWW_cW_cHW_UL2016': 166,'VBS_SSWW_DIM6_UL2016': 167,'VBS_SSWW_DIM6_SM_UL2016': 168,'ZZtoLep_UL2017': 169,'ZZTo2L2Nu_UL2017': 170,'ZZTo4L_UL2017': 171,'GluGluToContinToZZTo4e_UL2017': 172,'GluGluToContinToZZTo2e2mu_UL2017': 173,'GluGluToContinToZZTo2e2tau_UL2017': 174,'GluGluToContinToZZTo2mu2nu_UL2017': 175,'GluGluToContinToZZTo4mu_UL2017': 176,'GluGluToContinToZZTo2mu2tau_UL2017': 177,'GluGluToContinToZZTo2tau2nu_UL2017': 178,'GluGluToContinToZZTo4tau_UL2017': 179,'GluGluToContinToZZTo2e2nu_UL2017': 180,'TT_UL2017': 181,'TT_SemiLep_UL2017': 182,'TT_Had_UL2017': 183,'TTTo2L2Nu_UL2017': 184,'TT_beff_UL2017': 185,'TVX_UL2017': 186,'TTGJets_UL2017': 187,'TTZToQQ_UL2017': 188,'TTZToLLNuNu_UL2017': 189,'TTWJetsToQQ_UL2017': 190,'TTWJetsToLNu_UL2017': 191,'tZq_ll_4f_UL2017': 192,'VG_UL2017': 193,'ZG_UL2017': 194,'WG_UL2017': 195,'WrongSign_UL2017': 196,'WWto2L2Nu_UL2017': 197,'GluGluToWWToENEN_UL2017': 198,'GluGluToWWToENMN_UL2017': 199,'GluGluToWWToENTN_UL2017': 200,'GluGluToWWToMNEN_UL2017': 201,'GluGluToWWToMNMN_UL2017': 202,'GluGluToWWToMNTN_UL2017': 203,'GluGluToWWToTNEN_UL2017': 204,'GluGluToWWToTNMN_UL2017': 205,'GluGluToWWToTNTN_UL2017': 206,'ST_tW_top_UL2017': 207,'ST_tW_antitop_UL2017': 208,'GluGluHToWWTo2L2Nu_UL2017': 209,'GluGluHToWWToLNuQQ_UL2017': 210,'GluGluHToZZTo4L_UL2017': 211,'GluGluHToTauTau_UL2017': 212,'VBFHToWWTo2L2Nu_UL2017': 213,'VBFHToTauTau_UL2017': 214,'ttHToNonbb_UL2017': 215,'VHToNonbb_UL2017': 216,'Triboson_UL2017': 217,'WWTo2L2Nu_DoubleScattering_UL2017': 218,'WWW_4F_UL2017': 219,'WWZ_4F_UL2017': 220,'WZZ_UL2017': 221,'ZZZ_UL2017': 222,'WWG_UL2017': 223,'WJets_UL2017': 224,'WJetsHT70to100_UL2017': 225,'WJetsHT100to200_UL2017': 226,'WJetsHT200to400_UL2017': 227,'WJetsHT400to600_UL2017': 228,'WJetsHT600to800_UL2017': 229,'WJetsHT800to1200_UL2017': 230,'WJetsHT1200to2500_UL2017': 231,'WJetsHT2500toInf_UL2017': 232,'WZ_UL2017': 233,'DYJetsToLL_UL2017': 234,'DYJetsToLL_FxFx_UL2017': 234,'DYJetsToLL_M10to50_UL2017': 235,'DYJetsToLL_M50_UL2017': 236,'DYJetsToLL_M50_FxFx_UL2017': 236,'DYJetsToLL_M50_UL2017_ext': 236,'WpWpJJ_EWK_UL2017': 237,'WpWpJJ_QCD_UL2017': 238,'VBS_SSWW_SM_UL2017': 239,'VBS_SSWW_LL_SM_UL2017': 240,'VBS_SSWW_TL_SM_UL2017': 241,'VBS_SSWW_TT_SM_UL2017': 242,'VBS_SSWW_cW_UL2017': 243,'VBS_SSWW_cW_SM_UL2017': 244,'VBS_SSWW_cW_BSM_UL2017': 245,'VBS_SSWW_cW_INT_UL2017': 246,'VBS_SSWW_cHW_UL2017': 247,'VBS_SSWW_cHW_SM_UL2017': 248,'VBS_SSWW_cHW_BSM_UL2017': 249,'VBS_SSWW_cHW_INT_UL2017': 250,'VBS_SSWW_cW_cHW_UL2017': 251,'VBS_SSWW_DIM6_UL2017': 252,'VBS_SSWW_DIM6_SM_UL2017': 253,'ZZtoLep_UL2018': 254,'ZZTo2L2Nu_UL2018': 255,'ZZTo4L_UL2018': 256,'GluGluToContinToZZTo4e_UL2018': 257,'GluGluToContinToZZTo2e2mu_UL2018': 258,'GluGluToContinToZZTo2e2tau_UL2018': 259,'GluGluToContinToZZTo2mu2nu_UL2018': 260,'GluGluToContinToZZTo4mu_UL2018': 261,'GluGluToContinToZZTo2mu2tau_UL2018': 262,'GluGluToContinToZZTo2tau2nu_UL2018': 263,'GluGluToContinToZZTo4tau_UL2018': 264,'GluGluToContinToZZTo2e2nu_UL2018': 265,'TT_UL2018': 266,'TT_SemiLep_UL2018': 267,'TT_Had_UL2018': 268,'TTTo2L2Nu_UL2018': 269,'TT_beff_UL2018': 270,'TVX_UL2018': 271,'TTGJets_UL2018': 272,'TTZToQQ_UL2018': 273,'TTZToLLNuNu_UL2018': 274,'TTWJetsToQQ_UL2018': 275,'TTWJetsToLNu_UL2018': 276,'tZq_ll_4f_UL2018': 277,'VG_UL2018': 278,'ZG_UL2018': 279,'WG_UL2018': 280,'WrongSign_UL2018': 281,'WWto2L2Nu_UL2018': 282,'GluGluToWWToENEN_UL2018': 283,'GluGluToWWToENMN_UL2018': 284,'GluGluToWWToENTN_UL2018': 285,'GluGluToWWToMNEN_UL2018': 286,'GluGluToWWToMNMN_UL2018': 287,'GluGluToWWToMNTN_UL2018': 288,'GluGluToWWToTNEN_UL2018': 289,'GluGluToWWToTNMN_UL2018': 290,'GluGluToWWToTNTN_UL2018': 291,'ST_tW_top_UL2018': 292,'ST_tW_antitop_UL2018': 293,'GluGluHToWWTo2L2Nu_UL2018': 294,'GluGluHToWWToLNuQQ_UL2018': 295,'GluGluHToZZTo4L_UL2018': 296,'GluGluHToTauTau_UL2018': 297,'VBFHToWWTo2L2Nu_UL2018': 298,'VBFHToTauTau_UL2018': 299,'ttHToNonbb_UL2018': 300,'VHToNonbb_UL2018': 301,'Triboson_UL2018': 302,'WWTo2L2Nu_DoubleScattering_UL2018': 303,'WWW_4F_UL2018': 304,'WWZ_4F_UL2018': 305,'WZZ_UL2018': 306,'ZZZ_UL2018': 307,'WWG_UL2018': 308,'WJets_UL2018': 309,'WJetsHT70to100_UL2018': 310,'WJetsHT100to200_UL2018': 311,'WJetsHT200to400_UL2018': 312,'WJetsHT400to600_UL2018': 313,'WJetsHT600to800_UL2018': 314,'WJetsHT800to1200_UL2018': 315,'WJetsHT1200to2500_UL2018': 316,'WJetsHT2500toInf_UL2018': 317,'WZ_UL2018': 318,'DYJetsToLL_UL2018': 319,'DYJetsToLL_FxFx_UL2018': 319,'DYJetsToLL_M10to50_UL2018': 320,'DYJetsToLL_M50_UL2018': 321,'DYJetsToLL_M50_FxFx_UL2018': 321,'DYJetsToLL_M50_UL2018_ext': 321,'WpWpJJ_EWK_UL2018': 322,'WpWpJJ_QCD_UL2018': 323,'VBS_SSWW_SM_UL2018': 324,'VBS_SSWW_LL_SM_UL2018': 325,'VBS_SSWW_TL_SM_UL2018': 326,'VBS_SSWW_TT_SM_UL2018': 327,'VBS_SSWW_cW_UL2018': 328,'VBS_SSWW_cW_BSM_UL2018': 329,'VBS_SSWW_cW_SM_UL2018': 330,'VBS_SSWW_cW_INT_UL2018': 331,'VBS_SSWW_cHW_UL2018': 332,'VBS_SSWW_cHW_SM_UL2018': 333,'VBS_SSWW_cHW_BSM_UL2018': 334,'VBS_SSWW_cHW_INT_UL2018': 335,'VBS_SSWW_cW_cHW_UL2018': 336,'VBS_SSWW_DIM6_UL2018': 337,'VBS_SSWW_DIM6_SM_UL2018': 338,'DataMu_UL2016APV': 339,'DataMuB1_UL2016APV': 340,'DataMuB2_UL2016APV': 341,'DataMuC_UL2016APV': 342,'DataMuD_UL2016APV': 343,'DataMuE_UL2016APV': 344,'DataMuF_UL2016APV': 345,'DataMu_UL2016': 346,'DataMuF_UL2016': 347,'DataMuG_UL2016': 348,'DataMuH_UL2016': 349,'DataMu_UL2017': 350,'DataMuB_UL2017': 351,'DataMuC_UL2017': 352,'DataMuD_UL2017': 353,'DataMuE_UL2017': 354,'DataMuF_UL2017': 355,'DataMu_UL2018': 356,'DataMuA_UL2018': 357,'DataMuB_UL2018': 358,'DataMuC_UL2018': 359,'DataMuD_UL2018': 360,'DataEle_UL2016APV': 361,'DataEleB1_UL2016APV': 362,'DataEleB2_UL2016APV': 363,'DataEleC_UL2016APV': 364,'DataEleD_UL2016APV': 365,'DataEleE_UL2016APV': 366,'DataEleF_UL2016APV': 367,'DataEle_UL2016': 368,'DataEleF_UL2016': 369,'DataEleG_UL2016': 370,'DataEleH_UL2016': 371,'DataEle_UL2017': 372,'DataEleB_UL2017': 373,'DataEleC_UL2017': 374,'DataEleD_UL2017': 375,'DataEleE_UL2017': 376,'DataEleF_UL2017': 377,'DataEle_UL2018': 378,'DataEleA_UL2018': 379,'DataEleB_UL2018': 380,'DataEleC_UL2018': 381,'DataEleD_UL2018': 382,'DataHT_UL2016APV': 383,'DataHTB1_UL2016APV': 384,'DataHTB2_UL2016APV': 385,'DataHTC_UL2016APV': 386,'DataHTD_UL2016APV': 387,'DataHTE_UL2016APV': 388,'DataHTF_UL2016APV': 389,'DataHT_UL2016': 390,'DataHTF_UL2016': 391,'DataHTG_UL2016': 392,'DataHTH_UL2016': 393,'DataHT_UL2017': 394,'DataHTB_UL2017': 395,'DataHTC_UL2017': 396,'DataHTD_UL2017': 397,'DataHTE_UL2017': 398,'DataHTF_UL2017': 399,'DataHT_UL2018': 400,'DataHTA_UL2018': 401,'DataHTB_UL2018': 402,'DataHTC_UL2018': 403,'DataHTD_UL2018': 404,'SampleHTFake_UL2016APV': 405,'SampleHTFake_UL2016': 406,'SampleHTFake_UL2017': 407,'SampleHTFake_UL2018': 408,}


aggregated_samples_UL2016 = {
    'VG':  [ZG_UL2016, WG_UL2016],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2016],   
    'TVX': [TTGJets_UL2016, TTZToQQ_UL2016, TTZToLLNuNu_UL2016, TTWJetsToQQ_UL2016, TTWJetsToLNu_UL2016, tZq_ll_4f_UL2016], 
    'Triboson': [WWW_4F_UL2016, WWZ_4F_UL2016, WZZ_UL2016, ZZZ_UL2016, WWG_UL2016], 
    'TTTo2L2Nu': [TTTo2L2Nu_UL2016], 
    'WZ': [WZ_UL2016], 
    'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2016],
    'WrongSign': [WWto2L2Nu_UL2016, GluGluToWWToENEN_UL2016, GluGluToWWToENMN_UL2016, GluGluToWWToENTN_UL2016, GluGluToWWToMNEN_UL2016, GluGluToWWToMNMN_UL2016, GluGluToWWToMNTN_UL2016, GluGluToWWToTNEN_UL2016, GluGluToWWToTNMN_UL2016, GluGluToWWToTNTN_UL2016, ST_tW_top_UL2016, 
                  ST_tW_antitop_UL2016, 
                  GluGluHToWWTo2L2Nu_UL2016, GluGluHToZZTo4L_UL2016, GluGluHToTauTau_UL2016, VBFHToWWTo2L2Nu_UL2016, VBFHToTauTau_UL2016, ttHToNonbb_UL2016, VHToNonbb_UL2016], 
    'ZZtoLep': [ZZTo2L2Nu_UL2016, ZZTo4L_UL2016, GluGluToContinToZZTo2e2nu_UL2016, GluGluToContinToZZTo2e2mu_UL2016, GluGluToContinToZZTo2e2tau_UL2016, GluGluToContinToZZTo2mu2nu_UL2016, GluGluToContinToZZTo2mu2tau_UL2016, GluGluToContinToZZTo4e_UL2016, GluGluToContinToZZTo4mu_UL2016, GluGluToContinToZZTo4tau_UL2016], 
    'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2016, VBS_SSWW_TL_SM_UL2016, VBS_SSWW_TT_SM_UL2016], 
    'Data': [DataEleF_UL2016, DataEleG_UL2016, DataEleH_UL2016, DataMuF_UL2016, DataMuG_UL2016, DataMuH_UL2016,], 
}

aggregated_samples_UL2016APV = {
    'VG':  [ZG_UL2016APV, WG_UL2016APV],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2016APV],   
    'TVX': [TTGJets_UL2016APV, TTZToQQ_UL2016APV, TTZToLLNuNu_UL2016APV, TTWJetsToQQ_UL2016APV, TTWJetsToLNu_UL2016APV, tZq_ll_4f_UL2016APV], 
    'Triboson': [WWW_4F_UL2016APV, WWZ_4F_UL2016APV, WZZ_UL2016APV, ZZZ_UL2016APV, WWG_UL2016APV], 
    'TTTo2L2Nu': [TTTo2L2Nu_UL2016APV], 
    'WZ': [WZ_UL2016APV], 
    'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2016APV],
    'WrongSign': [WWto2L2Nu_UL2016APV, GluGluToWWToENEN_UL2016APV, GluGluToWWToENMN_UL2016APV, GluGluToWWToENTN_UL2016APV, GluGluToWWToMNEN_UL2016APV, GluGluToWWToMNMN_UL2016APV, GluGluToWWToMNTN_UL2016APV, GluGluToWWToTNEN_UL2016APV, GluGluToWWToTNMN_UL2016APV, GluGluToWWToTNTN_UL2016APV, ST_tW_top_UL2016APV, 
                  ST_tW_antitop_UL2016APV, 
                  GluGluHToWWTo2L2Nu_UL2016APV, GluGluHToZZTo4L_UL2016APV, GluGluHToTauTau_UL2016APV, VBFHToWWTo2L2Nu_UL2016APV, VBFHToTauTau_UL2016APV, ttHToNonbb_UL2016APV, VHToNonbb_UL2016APV], 
    'ZZtoLep': [ZZTo2L2Nu_UL2016APV, ZZTo4L_UL2016APV, GluGluToContinToZZTo2e2nu_UL2016APV, GluGluToContinToZZTo2e2mu_UL2016APV, GluGluToContinToZZTo2e2tau_UL2016APV, GluGluToContinToZZTo2mu2nu_UL2016APV, GluGluToContinToZZTo2mu2tau_UL2016APV, GluGluToContinToZZTo4e_UL2016APV, GluGluToContinToZZTo4mu_UL2016APV, GluGluToContinToZZTo4tau_UL2016APV], 
    'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2016APV, VBS_SSWW_TL_SM_UL2016APV, VBS_SSWW_TT_SM_UL2016APV], 
    'Data': [DataEleB2_UL2016APV, DataEleC_UL2016APV, DataEleD_UL2016APV, DataEleE_UL2016APV, DataEleF_UL2016APV, DataMuB2_UL2016APV, DataMuC_UL2016APV, DataMuD_UL2016APV, DataMuE_UL2016APV, DataMuF_UL2016APV], 
}

aggregated_samples_UL2017 = {
    #'VG':  [ZG_UL2017, WG_UL2017],
    ##'QCD_ssWW': [WpWpJJ_QCD_UL2017],   
    #'TVX': [TTGJets_UL2017, TTZToQQ_UL2017, TTZToLLNuNu_UL2017, TTWJetsToQQ_UL2017, TTWJetsToLNu_UL2017, tZq_ll_4f_UL2017], 
    #'Triboson': [WWW_4F_UL2017, WWZ_4F_UL2017, WZZ_UL2017, ZZZ_UL2017, WWG_UL2017], 
    #'TTTo2L2Nu': [TTTo2L2Nu_UL2017], 
    'WZ': [WZ_UL2017], 
    #'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2017],
    #'WrongSign': [WWto2L2Nu_UL2017, GluGluToWWToENEN_UL2017, GluGluToWWToENMN_UL2017, GluGluToWWToENTN_UL2017, GluGluToWWToMNEN_UL2017, GluGluToWWToMNMN_UL2017, GluGluToWWToMNTN_UL2017, GluGluToWWToTNEN_UL2017, GluGluToWWToTNMN_UL2017, GluGluToWWToTNTN_UL2017, ST_tW_top_UL2017, 
    #              ST_tW_antitop_UL2017, 
    #              GluGluHToWWTo2L2Nu_UL2017, GluGluHToZZTo4L_UL2017, GluGluHToTauTau_UL2017, VBFHToWWTo2L2Nu_UL2017, VBFHToTauTau_UL2017, ttHToNonbb_UL2017, VHToNonbb_UL2017], 
    #'ZZtoLep': [ZZTo2L2Nu_UL2017, ZZTo4L_UL2017, GluGluToContinToZZTo2e2nu_UL2017, GluGluToContinToZZTo2e2mu_UL2017, GluGluToContinToZZTo2e2tau_UL2017, GluGluToContinToZZTo2mu2nu_UL2017, GluGluToContinToZZTo2mu2tau_UL2017, GluGluToContinToZZTo4e_UL2017, GluGluToContinToZZTo4mu_UL2017, GluGluToContinToZZTo4tau_UL2017], 
    #'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2017, VBS_SSWW_TL_SM_UL2017, VBS_SSWW_TT_SM_UL2017], 
    #'Data': [DataEleB_UL2017, DataEleC_UL2017, DataEleD_UL2017, DataEleE_UL2017, DataEleF_UL2017, DataMuB_UL2017, DataMuC_UL2017, DataMuD_UL2017, DataMuE_UL2017, DataMuF_UL2017], 
}

aggregated_samples_UL2018 = {
    'VG':  [ZG_UL2018, WG_UL2018],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2018],   
    'TVX': [TTGJets_UL2018, TTZToQQ_UL2018, TTZToLLNuNu_UL2018, TTWJetsToQQ_UL2018, TTWJetsToLNu_UL2018, tZq_ll_4f_UL2018], 
    'Triboson': [WWW_4F_UL2018, WWZ_4F_UL2018, WZZ_UL2018, ZZZ_UL2018, WWG_UL2018], 
    'TTTo2L2Nu': [TTTo2L2Nu_UL2018], 
    'WZ': [WZ_UL2018], 
    'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2018],
    'WrongSign': [WWto2L2Nu_UL2018, GluGluToWWToENEN_UL2018, GluGluToWWToENMN_UL2018, GluGluToWWToENTN_UL2018, GluGluToWWToMNEN_UL2018, GluGluToWWToMNMN_UL2018, GluGluToWWToMNTN_UL2018, GluGluToWWToTNEN_UL2018, GluGluToWWToTNMN_UL2018, GluGluToWWToTNTN_UL2018, ST_tW_top_UL2018, 
                  ST_tW_antitop_UL2018, 
                  GluGluHToWWTo2L2Nu_UL2018, GluGluHToZZTo4L_UL2018, GluGluHToTauTau_UL2018, VBFHToWWTo2L2Nu_UL2018, VBFHToTauTau_UL2018, ttHToNonbb_UL2018, VHToNonbb_UL2018], 
    'ZZtoLep': [ZZTo2L2Nu_UL2018, ZZTo4L_UL2018, GluGluToContinToZZTo2e2nu_UL2018, GluGluToContinToZZTo2e2mu_UL2018, GluGluToContinToZZTo2e2tau_UL2018, GluGluToContinToZZTo2mu2nu_UL2018, GluGluToContinToZZTo2mu2tau_UL2018, GluGluToContinToZZTo4e_UL2018, GluGluToContinToZZTo4mu_UL2018, GluGluToContinToZZTo4tau_UL2018], 
    'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2018, VBS_SSWW_TL_SM_UL2018, VBS_SSWW_TT_SM_UL2018], 
    'Data': [DataEleA_UL2018, DataEleB_UL2018, DataEleC_UL2018, DataEleD_UL2018, DataMuA_UL2018, DataMuB_UL2018, DataMuC_UL2018, DataMuD_UL2018], 
}

aggregated_samples = aggregated_samples_UL2017

In [18]:
with open('d_UL.json', 'r') as fp:
    file_dict = json.load(fp)

In [19]:
chain = []
chain_data = []
proxy = redirector
max_lines = maxNfilespersample
'''
for k,v in aggregated_samples.items():
    for s in v:
        files = file_dict[s.label] 
        for i,line in enumerate(files):
            if i < max_lines:
                if k != 'Data':
                    chain.append("root://" + proxy + "/" + line)
                else:
                    chain_data.append("root://" + proxy + "/" + line)
            i += 1
'''

# Get file replicas at desired site

for k,v in aggregated_samples.items():
    for s in v:
        #files = file_dict[s.label] 
        CMS_DATASET_NAME = s.dataset
        #print(CMS_DATASET_NAME)
        
        files = c.list_files(scope=RUCIO_SCOPE, name=CMS_DATASET_NAME)
        #print(files)
        lfns = []
        for f in files:
            lfns.append(RUCIO_SCOPE + ':' + f['name'])
        #print(lfns)
        #pfns_dicts = c.lfns2pfns(SITENAME, lfns, scheme='davs', operation='read')
        #pfns = []
        #for _,i in pfns_dicts.items():
        #    pfns.append(i)
        for i,line in enumerate(lfns):
            if i < max_lines and not line[4:] in [
                # "/store/mc/RunIISummer20UL17NanoAODv9/VBFHToWWTo2L2Nu_M-125_TuneCP5_13TeV-powheg-jhugen727-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v2/40000/002D8E22-0131-E84D-9E1B-F71AE8DB28C7.root",
                #"/store/mc/RunIISummer20UL17NanoAODv9/ST_tW_antitop_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v2/2430000/4ABEF3B8-7699-EA4C-9155-FFCE151E5F67.root",
                #"/store/mc/RunIISummer20UL17NanoAODv9/ST_tW_antitop_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v2/2430000/EDC6FA95-CDFD-CD49-942F-14434F37530A.root"
            ]:
                if k != 'Data':
                    chain.append("root://" + redirector + "/" + line[4:])
                    #chain.append("root://t2-xrdcms.lnl.infn.it:7070/" + line[4:])
                    #chain.append("davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms" + line[4:])
                else:
                    chain_data.append("root://" + redirector + "/" + line[4:])
                    #chain_data.append("root://t2-xrdcms.lnl.infn.it:7070/" + line[4:])
                    #chain_data.append("davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms" + line[4:])
            i += 1



#print(pfns)
print(len(chain))


20


In [20]:
chain

['root://cmsrm-xrootd02.roma1.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/WZ_TuneCP5_13TeV-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/936C5557-15D6-B74C-9260-582B71C8B1C3.root',
 'root://cmsrm-xrootd02.roma1.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/WZ_TuneCP5_13TeV-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AA222480-64FA-2140-B920-4606F5772B4C.root',
 'root://cmsrm-xrootd02.roma1.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/WZ_TuneCP5_13TeV-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/5CDC101C-4B0C-D248-B7DE-D3A4603F2D67.root',
 'root://cmsrm-xrootd02.roma1.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/WZ_TuneCP5_13TeV-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/A474C122-CA95-614C-9B59-9B59E4434863.root',
 'root://cmsrm-xrootd02.roma1.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/WZ_TuneCP5_13TeV-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/270000/57183F7B-DFD4-2F4A-8D0E-30146B8180B3.root',
 'root://cmsrm-xrootd02.rom

In [ ]:
ERROR GENERATOR

In [ ]:
#import ROOT
#f = ROOT.TFile.Open("root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/ST_tW_antitop_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v2/2430000/EDC6FA95-CDFD-CD49-942F-14434F37530A.root")
#f.ls()

In [ ]:
branches = [
    "nElectron","Electron_pt", "Electron_eta", "Electron_phi", "Electron_mass", "Electron_charge", "Electron_pdgId", "Electron_jetRelIso", "Electron_mvaFall17V2Iso_WPL", "Electron_mvaFall17V2Iso_WP90", 
    "Electron_effSF", "Electron_effSF_errUp", "Electron_effSF_errDown",
    "nMuon","Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_charge", "Muon_pdgId", "Muon_tightId", "Muon_looseId", "Muon_pfRelIso04_all", 
    "Muon_effSF", "Muon_effSF_errUp", "Muon_effSF_errDown", 
    "Muon_corrected_pt", "Muon_correctedUp_pt", "Muon_correctedDown_pt",
    "nTau","Tau_pt", "Tau_eta", "Tau_phi", "Tau_mass", "Tau_charge", "Tau_leadTkPtOverTauPt", "Tau_decayMode", "Tau_neutralIso", "Tau_idDeepTau2017v2p1VSjet", "Tau_idDeepTau2017v2p1VSe", "Tau_idDeepTau2017v2p1VSmu", "Tau_genPartFlav",
    "nJet","Jet_pt", "Jet_eta", "Jet_phi", "Jet_mass", "Jet_jetId", "Jet_puId", "Jet_btagDeepFlavB", "Jet_btagDeepB", "Jet_btagCSVV2", "Jet_partonFlavour", 
    "Jet_btagSF_deepjet_M_up", "Jet_btagSF_deepjet_M_down", "Jet_btagSF_deepjet_M",
    #"nFatJet",
    "PrefireWeight", "PrefireWeightUp", "PrefireWeightDown", 
    "puWeight", "puWeightUp", "puWeightDown", 
    "Sample",
    "Jet_pt_nom", "Jet_mass_nom", 
    "FatJet_pt_nom", "FatJet_mass_nom", "FatJet_msoftdrop_nom", 
    "MET_T1_pt_nom", "MET_T1_phi_nom", 
    "MET_T1Smear_pt_nom", "MET_T1Smear_phi_nom",
    "MET_pt", "MET_phi",
]


#variations = ['nominal', 'jesTotalup', 'jesTotaldown','jerup', 'jerdown']
variations = jetvariations

for n,v in enumerate(jetvariations[1:]):
    if v in variations:
        branches.append("Jet_pt_{}".format(v))
        branches.append("Jet_mass_{}".format(v))
for n,v in enumerate(fatjetvariations[1:]):
    if v in variations:
        branches.append("FatJet_pt_{}".format(v))
        branches.append("FatJet_mass_{}".format(v))
        branches.append("FatJet_msoftdrop_{}".format(v))
for n,v in enumerate(mett1variations[1:]):
    if v in variations:
        branches.append("MET_T1_pt_{}".format(v))
        branches.append("MET_T1_phi_{}".format(v))  
for n,v in enumerate(mett1smearvariations[1:]):
    if v in variations:
        branches.append("MET_T1Smear_pt_{}".format(v))
        branches.append("MET_T1Smear_phi_{}".format(v))  

#branches = ["nElectron","Electron_pt"] for debug

print("branches:", branches)

branchlist = ROOT.vector('string')()
for branchName in branches:
    branchlist.push_back(branchName)

## Define the dataframe starting from a chain of root files

In [ ]:
#chain = ["root://" + proxy + "//store/mc/RunIIFall17NanoAODv7/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/100000/5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root"]
 #VBS_SSWW_LL_2017
#chain = ["root://stormgf2.pi.infn.it//store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/9E8D16F0-7606-A84D-B0A2-D322047D05D8.root"]
'''
chain = [
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/1FD30052-F2C4-F641-A385-CD4045C5EF69.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/561AC64C-2D57-EF4C-B6A4-7804766544F9.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/708B7CC3-30A4-FA47-B6ED-3CCC018F5AE5.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/AECF07BB-886E-8F47-823D-458E2EBA6FFD.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B8F2CB7B-450E-F242-8F3C-7655DB1DC0C5.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/F1A0BF26-0EEF-344F-B0F9-9A655DB39075.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/130000/C8F09A2D-A515-A243-A4E7-C71726EA1714.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/270000/31D0A4FB-4AD9-774A-82E7-5C00DC8CCB91.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/270000/75A8EA12-95BC-4848-BB6E-D1E347BC2750.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/270000/998E17BC-A2F9-F54C-B9A5-BC5209289E1B.root',
    'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/270000/99F6CA79-CA53-FC45-810F-D0C1CDBEEEC2.root',
]
'''
'''
chain = [
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/35576172-00C8-5D42-977A-D53B4847DD4A.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/3851571D-5761-6544-8AD9-5E0BF62EE3EB.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/39FAA3C8-01C8-0247-B4EF-849957BBF4D8.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/3D9CF396-C486-E54D-8896-3614F421C0A6.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/3DE27AC4-666E-814E-81E4-FAEA41E27A70.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/47C6CCFD-2B70-994C-97D4-3EB9FEBC4C4E.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/4C1E172E-DA11-8C4E-B59D-0B3A46A8956B.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/60F46F09-86AE-B541-8A5A-DEB0A29A8D50.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/72DE1A8F-B8E6-9C4E-A073-02C6036C7B00.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/74AEE4CE-B242-4242-9AD9-382EEEFB223C.root',
'root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2510000/85C9F160-4573-7E44-8594-5D156977A2B1.root',
]
'''
#chain = [
#"davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms/store/mc/RunIISummer20UL17NanoAODv9/VBS_SSWW_TT_polarization_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/2430000/18D73379-5567-BC44-8A79-1E8053536AA7.root"
#]



In [ ]:
#columns = [
#    "CaloMET_phi", "CaloMET_pt", "CaloMET_sumEt", "ChsMET_phi", "ChsMET_pt", "ChsMET_sumEt", "CorrT1METJet_area", "CorrT1METJet_eta", "CorrT1METJet_muonSubtrFactor", "CorrT1METJet_phi", "CorrT1METJet_rawPt", "DeepMETResolutionTune_phi", "DeepMETResolutionTune_pt", "DeepMETResponseTune_phi", "DeepMETResponseTune_pt", "Electron_charge", "Electron_cleanmask", "Electron_convVeto", "Electron_cutBased", "Electron_cutBased_HEEP", "Electron_dEscaleDown", "Electron_dEscaleUp", "Electron_dEsigmaDown", "Electron_dEsigmaUp", "Electron_deltaEtaSC", "Electron_dr03EcalRecHitSumEt", "Electron_dr03HcalDepth1TowerSumEt", "Electron_dr03TkSumPt", "Electron_dr03TkSumPtHEEP", "Electron_dxy", "Electron_dxyErr", "Electron_dz", "Electron_dzErr", "Electron_eCorr", "Electron_eInvMinusPInv", "Electron_energyErr", "Electron_eta", "Electron_genPartFlav", "Electron_genPartIdx", "Electron_hoe", "Electron_ip3d", "Electron_isPFcand", "Electron_jetIdx", "Electron_jetNDauCharged", "Electron_jetPtRelv2", "Electron_jetRelIso", "Electron_lostHits", "Electron_mass", "Electron_miniPFRelIso_all", "Electron_miniPFRelIso_chg", "Electron_mvaFall17V2Iso", "Electron_mvaFall17V2Iso_WP80", "Electron_mvaFall17V2Iso_WP90", "Electron_mvaFall17V2Iso_WPL", "Electron_mvaFall17V2noIso", "Electron_mvaFall17V2noIso_WP80", "Electron_mvaFall17V2noIso_WP90", "Electron_mvaFall17V2noIso_WPL", "Electron_mvaTTH", "Electron_pdgId", "Electron_pfRelIso03_all", "Electron_pfRelIso03_chg", "Electron_phi", "Electron_photonIdx", "Electron_pt", "Electron_r9", "Electron_scEtOverPt", "Electron_seedGain", "Electron_sieie", "Electron_sip3d", "Electron_tightCharge", "Electron_vidNestedWPBitmap", "Electron_vidNestedWPBitmapHEEP", "FatJet_area", "FatJet_btagCSVV2", "FatJet_btagDDBvLV2", "FatJet_btagDDCvBV2", "FatJet_btagDDCvLV2", "FatJet_btagDeepB", "FatJet_btagHbb", "FatJet_deepTagMD_H4qvsQCD", "FatJet_deepTagMD_HbbvsQCD", "FatJet_deepTagMD_TvsQCD", "FatJet_deepTagMD_WvsQCD", "FatJet_deepTagMD_ZHbbvsQCD", "FatJet_deepTagMD_ZHccvsQCD", "FatJet_deepTagMD_ZbbvsQCD", "FatJet_deepTagMD_ZvsQCD", "FatJet_deepTagMD_bbvsLight", "FatJet_deepTagMD_ccvsLight", "FatJet_deepTag_H", "FatJet_deepTag_QCD", "FatJet_deepTag_QCDothers", "FatJet_deepTag_TvsQCD", "FatJet_deepTag_WvsQCD", "FatJet_deepTag_ZvsQCD", "FatJet_electronIdx3SJ", "FatJet_eta", "FatJet_genJetAK8Idx", "FatJet_hadronFlavour", "FatJet_jetId", "FatJet_lsf3", "FatJet_mass", "FatJet_msoftdrop", "FatJet_muonIdx3SJ", "FatJet_n2b1", "FatJet_n3b1", "FatJet_nBHadrons", "FatJet_nCHadrons", "FatJet_nConstituents", "FatJet_particleNetMD_QCD", "FatJet_particleNetMD_Xbb", "FatJet_particleNetMD_Xcc", "FatJet_particleNetMD_Xqq", "FatJet_particleNet_H4qvsQCD", "FatJet_particleNet_HbbvsQCD", "FatJet_particleNet_HccvsQCD", "FatJet_particleNet_QCD", "FatJet_particleNet_TvsQCD", "FatJet_particleNet_WvsQCD", "FatJet_particleNet_ZvsQCD", "FatJet_particleNet_mass", "FatJet_phi", "FatJet_pt", "FatJet_rawFactor", "FatJet_subJetIdx1", "FatJet_subJetIdx2", "FatJet_tau1", "FatJet_tau2", "FatJet_tau3", "FatJet_tau4", "Flag_BadChargedCandidateFilter", "Flag_BadChargedCandidateSummer16Filter", "Flag_BadPFMuonDzFilter", "Flag_BadPFMuonFilter", "Flag_BadPFMuonSummer16Filter", "Flag_CSCTightHalo2015Filter", "Flag_CSCTightHaloFilter", "Flag_CSCTightHaloTrkMuUnvetoFilter", "Flag_EcalDeadCellBoundaryEnergyFilter", "Flag_EcalDeadCellTriggerPrimitiveFilter", "Flag_HBHENoiseFilter", "Flag_HBHENoiseIsoFilter", "Flag_HcalStripHaloFilter", "Flag_METFilters", "Flag_chargedHadronTrackResolutionFilter", "Flag_ecalBadCalibFilter", "Flag_ecalLaserCorrFilter", "Flag_eeBadScFilter", "Flag_globalSuperTightHalo2016Filter", "Flag_globalTightHalo2016Filter", "Flag_goodVertices", "Flag_hcalLaserEventFilter", "Flag_hfNoisyHitsFilter", "Flag_muonBadTrackFilter", "Flag_trkPOGFilters", "Flag_trkPOG_logErrorTooManyClusters", "Flag_trkPOG_manystripclus53X", "Flag_trkPOG_toomanystripclus53X", "FsrPhoton_dROverEt2", "FsrPhoton_eta", "FsrPhoton_muonIdx", "FsrPhoton_phi", "FsrPhoton_pt", "FsrPhoton_relIso03", "GenDressedLepton_eta", "GenDressedLepton_hasTauAnc", "GenDressedLepton_mass", "GenDressedLepton_pdgId", "GenDressedLepton_phi", "GenDressedLepton_pt", "GenIsolatedPhoton_eta", "GenIsolatedPhoton_mass", "GenIsolatedPhoton_phi", "GenIsolatedPhoton_pt", "GenJetAK8_eta", "GenJetAK8_hadronFlavour", "GenJetAK8_mass", "GenJetAK8_partonFlavour", "GenJetAK8_phi", "GenJetAK8_pt", "GenJet_eta", "GenJet_hadronFlavour", "GenJet_mass", "GenJet_partonFlavour", "GenJet_phi", "GenJet_pt", "GenMET_phi", "GenMET_pt", "GenPart_eta", "GenPart_genPartIdxMother", "GenPart_mass", "GenPart_pdgId", "GenPart_phi", "GenPart_pt", "GenPart_status", "GenPart_statusFlags", "GenVisTau_charge", "GenVisTau_eta", "GenVisTau_genPartIdxMother", "GenVisTau_mass", "GenVisTau_phi", "GenVisTau_pt", "GenVisTau_status", "GenVtx_t0", "GenVtx_x", "GenVtx_y", "GenVtx_z", "Generator_binvar", "Generator_id1", "Generator_id2", "Generator_scalePDF", "Generator_weight", "Generator_x1", "Generator_x2", "Generator_xpdf1", "Generator_xpdf2", "HLT_AK4CaloJet100", "HLT_AK4CaloJet120", "HLT_AK4CaloJet30", "HLT_AK4CaloJet40", "HLT_AK4CaloJet50", "HLT_AK4CaloJet80", "HLT_AK4PFJet100", "HLT_AK4PFJet120", "HLT_AK4PFJet30", "HLT_AK4PFJet50", "HLT_AK4PFJet80", "HLT_AK8PFHT750_TrimMass50", "HLT_AK8PFHT800_TrimMass50", "HLT_AK8PFHT850_TrimMass50", "HLT_AK8PFHT900_TrimMass50", "HLT_AK8PFJet140", "HLT_AK8PFJet200", "HLT_AK8PFJet260", "HLT_AK8PFJet320", "HLT_AK8PFJet330_PFAK8BTagCSV_p1", "HLT_AK8PFJet330_PFAK8BTagCSV_p17", "HLT_AK8PFJet360_TrimMass30", "HLT_AK8PFJet380_TrimMass30", "HLT_AK8PFJet40", "HLT_AK8PFJet400", "HLT_AK8PFJet400_TrimMass30", "HLT_AK8PFJet420_TrimMass30", "HLT_AK8PFJet450", "HLT_AK8PFJet500", "HLT_AK8PFJet550", "HLT_AK8PFJet60", "HLT_AK8PFJet80", "HLT_AK8PFJetFwd140", "HLT_AK8PFJetFwd200", "HLT_AK8PFJetFwd260", "HLT_AK8PFJetFwd320", "HLT_AK8PFJetFwd40", "HLT_AK8PFJetFwd400", "HLT_AK8PFJetFwd450", "HLT_AK8PFJetFwd500", "HLT_AK8PFJetFwd60", "HLT_AK8PFJetFwd80", "HLT_BTagMu_AK4DiJet110_Mu5", "HLT_BTagMu_AK4DiJet170_Mu5", "HLT_BTagMu_AK4DiJet20_Mu5", "HLT_BTagMu_AK4DiJet40_Mu5", "HLT_BTagMu_AK4DiJet70_Mu5", "HLT_BTagMu_AK4Jet300_Mu5", "HLT_BTagMu_AK8DiJet170_Mu5", "HLT_BTagMu_AK8Jet300_Mu5", "HLT_CaloJet500_NoJetID", "HLT_CaloJet550_NoJetID", "HLT_CaloMET100_HBHECleaned", "HLT_CaloMET100_NotCleaned", "HLT_CaloMET110_NotCleaned", "HLT_CaloMET250_HBHECleaned", "HLT_CaloMET250_NotCleaned", "HLT_CaloMET300_HBHECleaned", "HLT_CaloMET350_HBHECleaned", "HLT_CaloMET70_HBHECleaned", "HLT_CaloMET80_HBHECleaned", "HLT_CaloMET80_NotCleaned", "HLT_CaloMET90_HBHECleaned", "HLT_CaloMET90_NotCleaned", "HLT_CaloMHT90", "HLT_DiEle27_WPTightCaloOnly_L1DoubleEG", "HLT_DiJet110_35_Mjj650_PFMET110", "HLT_DiJet110_35_Mjj650_PFMET120", "HLT_DiJet110_35_Mjj650_PFMET130", "HLT_DiMu9_Ele9_CaloIdL_TrackIdL", "HLT_DiMu9_Ele9_CaloIdL_TrackIdL_DZ", "HLT_DiPFJet15_FBEta3_NoCaloMatched", "HLT_DiPFJet15_NoCaloMatched", "HLT_DiPFJet25_FBEta3_NoCaloMatched", "HLT_DiPFJet25_NoCaloMatched", "HLT_DiPFJetAve100_HFJEC", "HLT_DiPFJetAve140", "HLT_DiPFJetAve15_HFJEC", "HLT_DiPFJetAve160_HFJEC", "HLT_DiPFJetAve200", "HLT_DiPFJetAve220_HFJEC", "HLT_DiPFJetAve25_HFJEC", "HLT_DiPFJetAve260", "HLT_DiPFJetAve300_HFJEC", "HLT_DiPFJetAve320", "HLT_DiPFJetAve35_HFJEC", "HLT_DiPFJetAve40", "HLT_DiPFJetAve400", "HLT_DiPFJetAve500", "HLT_DiPFJetAve60", "HLT_DiPFJetAve60_HFJEC", "HLT_DiPFJetAve80", "HLT_DiPFJetAve80_HFJEC", "HLT_DiSC30_18_EIso_AND_HE_Mass70", "HLT_Dimuon0_Jpsi", "HLT_Dimuon0_Jpsi3p5_Muon2", "HLT_Dimuon0_Jpsi_L1_4R_0er1p5R", "HLT_Dimuon0_Jpsi_L1_NoOS", "HLT_Dimuon0_Jpsi_NoVertexing", "HLT_Dimuon0_Jpsi_NoVertexing_L1_4R_0er1p5R", "HLT_Dimuon0_Jpsi_NoVertexing_NoOS", "HLT_Dimuon0_LowMass", "HLT_Dimuon0_LowMass_L1_0er1p5", "HLT_Dimuon0_LowMass_L1_0er1p5R", "HLT_Dimuon0_LowMass_L1_4", "HLT_Dimuon0_LowMass_L1_4R", "HLT_Dimuon0_LowMass_L1_TM530", "HLT_Dimuon0_Upsilon_L1_4p5", "HLT_Dimuon0_Upsilon_L1_4p5NoOS", "HLT_Dimuon0_Upsilon_L1_4p5er2p0", "HLT_Dimuon0_Upsilon_L1_4p5er2p0M", "HLT_Dimuon0_Upsilon_L1_5", "HLT_Dimuon0_Upsilon_L1_5M", "HLT_Dimuon0_Upsilon_Muon_L1_TM0", "HLT_Dimuon0_Upsilon_Muon_NoL1Mass", "HLT_Dimuon0_Upsilon_NoVertexing", "HLT_Dimuon10_PsiPrime_Barrel_Seagulls", "HLT_Dimuon10_Upsilon_Barrel_Seagulls", "HLT_Dimuon12_Upsilon_eta1p5", "HLT_Dimuon14_Phi_Barrel_Seagulls", "HLT_Dimuon18_PsiPrime", "HLT_Dimuon18_PsiPrime_noCorrL1", "HLT_Dimuon20_Jpsi_Barrel_Seagulls", "HLT_Dimuon24_Phi_noCorrL1", "HLT_Dimuon24_Upsilon_noCorrL1", "HLT_Dimuon25_Jpsi", "HLT_Dimuon25_Jpsi_noCorrL1", "HLT_Diphoton30EB_18EB_R9Id_OR_IsoCaloId_AND_HE_R9Id_NoPixelVeto_Mass55", "HLT_Diphoton30EB_18EB_R9Id_OR_IsoCaloId_AND_HE_R9Id_PixelVeto_Mass55", "HLT_Diphoton30PV_18PV_R9Id_AND_IsoCaloId_AND_HE_R9Id_NoPixelVeto_Mass55", "HLT_Diphoton30PV_18PV_R9Id_AND_IsoCaloId_AND_HE_R9Id_PixelVeto_Mass55", "HLT_Diphoton30_18_PVrealAND_R9Id_AND_IsoCaloId_AND_HE_R9Id_NoPixelVeto_Mass55", "HLT_Diphoton30_18_PVrealAND_R9Id_AND_IsoCaloId_AND_HE_R9Id_PixelVeto_Mass55", "HLT_Diphoton30_22_R9Id_OR_IsoCaloId_AND_HE_R9Id_Mass90", "HLT_Diphoton30_22_R9Id_OR_IsoCaloId_AND_HE_R9Id_Mass95", "HLT_DoubleEle24_eta2p1_WPTight_Gsf", "HLT_DoubleEle25_CaloIdL_MW", "HLT_DoubleEle27_CaloIdL_MW", "HLT_DoubleEle33_CaloIdL_MW", "HLT_DoubleEle8_CaloIdM_TrackIdM_Mass8_DZ_PFHT350", "HLT_DoubleEle8_CaloIdM_TrackIdM_Mass8_PFHT350", "HLT_DoubleIsoMu20_eta2p1", "HLT_DoubleIsoMu24_eta2p1", "HLT_DoubleL2Mu50", "HLT_DoubleLooseChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg", "HLT_DoubleLooseChargedIsoPFTau35_Trk1_eta2p1_Reg", "HLT_DoubleLooseChargedIsoPFTau40_Trk1_TightID_eta2p1_Reg", "HLT_DoubleLooseChargedIsoPFTau40_Trk1_eta2p1_Reg", "HLT_DoubleMediumChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg", "HLT_DoubleMediumChargedIsoPFTau35_Trk1_eta2p1_Reg", "HLT_DoubleMediumChargedIsoPFTau40_Trk1_TightID_eta2p1_Reg", "HLT_DoubleMediumChargedIsoPFTau40_Trk1_eta2p1_Reg", "HLT_DoubleMu20_7_Mass0to30_L1_DM4", "HLT_DoubleMu20_7_Mass0to30_L1_DM4EG", "HLT_DoubleMu20_7_Mass0to30_Photon23", "HLT_DoubleMu2_Jpsi_DoubleTkMu0_Phi", "HLT_DoubleMu2_Jpsi_DoubleTrk1_Phi", "HLT_DoubleMu3_DCA_PFMET50_PFMHT60", "HLT_DoubleMu3_DZ_PFMET50_PFMHT60", "HLT_DoubleMu3_DZ_PFMET70_PFMHT70", "HLT_DoubleMu3_DZ_PFMET90_PFMHT90", "HLT_DoubleMu3_Trk_Tau3mu", "HLT_DoubleMu3_Trk_Tau3mu_NoL1Mass", "HLT_DoubleMu43NoFiltersNoVtx", "HLT_DoubleMu48NoFiltersNoVtx", "HLT_DoubleMu4_3_Bs", "HLT_DoubleMu4_3_Jpsi_Displaced", "HLT_DoubleMu4_JpsiTrkTrk_Displaced", "HLT_DoubleMu4_JpsiTrk_Displaced", "HLT_DoubleMu4_Jpsi_Displaced", "HLT_DoubleMu4_Jpsi_NoVertexing", "HLT_DoubleMu4_LowMassNonResonantTrk_Displaced", "HLT_DoubleMu4_Mass8_DZ_PFHT350", "HLT_DoubleMu4_PsiPrimeTrk_Displaced", "HLT_DoubleMu8_Mass8_PFHT350", "HLT_DoublePFJets100MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_DoublePFJets100_CaloBTagCSV_p33", "HLT_DoublePFJets116MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_DoublePFJets128MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_DoublePFJets200_CaloBTagCSV_p33", "HLT_DoublePFJets350_CaloBTagCSV_p33", "HLT_DoublePFJets40_CaloBTagCSV_p33", "HLT_DoublePhoton33_CaloIdL", "HLT_DoublePhoton70", "HLT_DoublePhoton85", "HLT_DoubleTightChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg", "HLT_DoubleTightChargedIsoPFTau35_Trk1_eta2p1_Reg", "HLT_DoubleTightChargedIsoPFTau40_Trk1_TightID_eta2p1_Reg", "HLT_DoubleTightChargedIsoPFTau40_Trk1_eta2p1_Reg", "HLT_ECALHT800", "HLT_EcalCalibration", "HLT_Ele115_CaloIdVT_GsfTrkIdT", "HLT_Ele12_CaloIdL_TrackIdL_IsoVL_PFJet30", "HLT_Ele135_CaloIdVT_GsfTrkIdT", "HLT_Ele145_CaloIdVT_GsfTrkIdT", "HLT_Ele15_IsoVVVL_PFHT450", "HLT_Ele15_IsoVVVL_PFHT450_CaloBTagCSV_4p5", "HLT_Ele15_IsoVVVL_PFHT450_PFMET50", "HLT_Ele15_IsoVVVL_PFHT600", "HLT_Ele16_Ele12_Ele8_CaloIdL_TrackIdL", "HLT_Ele17_CaloIdM_TrackIdM_PFJet30", "HLT_Ele200_CaloIdVT_GsfTrkIdT", "HLT_Ele20_WPLoose_Gsf", "HLT_Ele20_WPTight_Gsf", "HLT_Ele20_eta2p1_WPLoose_Gsf", "HLT_Ele23_CaloIdL_TrackIdL_IsoVL_PFJet30", "HLT_Ele23_CaloIdM_TrackIdM_PFJet30", "HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL", "HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL_DZ", "HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau30_eta2p1_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau30_eta2p1_TightID_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_MediumChargedIsoPFTau30_eta2p1_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_MediumChargedIsoPFTau30_eta2p1_TightID_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_TightChargedIsoPFTau30_eta2p1_CrossL1", "HLT_Ele24_eta2p1_WPTight_Gsf_TightChargedIsoPFTau30_eta2p1_TightID_CrossL1", "HLT_Ele250_CaloIdVT_GsfTrkIdT", "HLT_Ele27_Ele37_CaloIdL_MW", "HLT_Ele27_WPTight_Gsf", "HLT_Ele28_HighEta_SC20_Mass55", "HLT_Ele28_eta2p1_WPTight_Gsf_HT150", "HLT_Ele300_CaloIdVT_GsfTrkIdT", "HLT_Ele30_eta2p1_WPTight_Gsf_CentralPFJet35_EleCleaned", "HLT_Ele32_WPTight_Gsf", "HLT_Ele32_WPTight_Gsf_L1DoubleEG", "HLT_Ele35_WPTight_Gsf", "HLT_Ele35_WPTight_Gsf_L1EGMT", "HLT_Ele38_WPTight_Gsf", "HLT_Ele40_WPTight_Gsf", "HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165", "HLT_Ele50_IsoVVVL_PFHT450", "HLT_Ele8_CaloIdL_TrackIdL_IsoVL_PFJet30", "HLT_Ele8_CaloIdM_TrackIdM_PFJet30", "HLT_FullTrack_Multiplicity100", "HLT_FullTrack_Multiplicity130", "HLT_FullTrack_Multiplicity155", "HLT_FullTrack_Multiplicity85", "HLT_HISinglePhoton10_Eta3p1ForPPRef", "HLT_HISinglePhoton20_Eta3p1ForPPRef", "HLT_HISinglePhoton30_Eta3p1ForPPRef", "HLT_HISinglePhoton40_Eta3p1ForPPRef", "HLT_HISinglePhoton50_Eta3p1ForPPRef", "HLT_HISinglePhoton60_Eta3p1ForPPRef", "HLT_HT300_Beamspot", "HLT_HT400_DisplacedDijet40_DisplacedTrack", "HLT_HT425", "HLT_HT430_DisplacedDijet40_DisplacedTrack", "HLT_HT430_DisplacedDijet60_DisplacedTrack", "HLT_HT430_DisplacedDijet80_DisplacedTrack", "HLT_HT450_Beamspot", "HLT_HT550_DisplacedDijet60_Inclusive", "HLT_HT550_DisplacedDijet80_Inclusive", "HLT_HT650_DisplacedDijet60_Inclusive", "HLT_HT650_DisplacedDijet80_Inclusive", "HLT_HT750_DisplacedDijet80_Inclusive", "HLT_HcalCalibration", "HLT_HcalIsolatedbunch", "HLT_HcalNZS", "HLT_HcalPhiSym", "HLT_IsoMu20", "HLT_IsoMu20_eta2p1_LooseChargedIsoPFTau27_eta2p1_CrossL1", "HLT_IsoMu20_eta2p1_LooseChargedIsoPFTau27_eta2p1_TightID_CrossL1", "HLT_IsoMu20_eta2p1_MediumChargedIsoPFTau27_eta2p1_CrossL1", "HLT_IsoMu20_eta2p1_MediumChargedIsoPFTau27_eta2p1_TightID_CrossL1", "HLT_IsoMu20_eta2p1_TightChargedIsoPFTau27_eta2p1_CrossL1", "HLT_IsoMu20_eta2p1_TightChargedIsoPFTau27_eta2p1_TightID_CrossL1", "HLT_IsoMu24", "HLT_IsoMu24_eta2p1", "HLT_IsoMu24_eta2p1_LooseChargedIsoPFTau20_SingleL1", "HLT_IsoMu24_eta2p1_LooseChargedIsoPFTau20_TightID_SingleL1", "HLT_IsoMu24_eta2p1_LooseChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_LooseChargedIsoPFTau35_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau20_SingleL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau20_TightID_SingleL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau35_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau40_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau40_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_MediumChargedIsoPFTau50_Trk30_eta2p1_1pr", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau20_SingleL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau20_TightID_SingleL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau35_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau35_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau40_Trk1_TightID_eta2p1_Reg_CrossL1", "HLT_IsoMu24_eta2p1_TightChargedIsoPFTau40_Trk1_eta2p1_Reg_CrossL1", "HLT_IsoMu27", "HLT_IsoMu27_LooseChargedIsoPFTau20_SingleL1", "HLT_IsoMu27_MediumChargedIsoPFTau20_SingleL1", "HLT_IsoMu27_TightChargedIsoPFTau20_SingleL1", "HLT_IsoMu30", "HLT_IsoTrackHB", "HLT_IsoTrackHE", "HLT_L1ETMHadSeeds", "HLT_L1MinimumBiasHF0OR", "HLT_L1MinimumBiasHF_OR", "HLT_L1NotBptxOR", "HLT_L1SingleMu18", "HLT_L1SingleMu25", "HLT_L1UnpairedBunchBptxMinus", "HLT_L1UnpairedBunchBptxPlus", "HLT_L1_CDC_SingleMu_3_er1p2_TOP120_DPHI2p618_3p142", "HLT_L1_DoubleJet30_Mass_Min400_Mu10", "HLT_L2Mu10", "HLT_L2Mu10_NoVertex_NoBPTX", "HLT_L2Mu10_NoVertex_NoBPTX3BX", "HLT_L2Mu40_NoVertex_3Sta_NoBPTX3BX", "HLT_L2Mu45_NoVertex_3Sta_NoBPTX3BX", "HLT_L2Mu50", "HLT_MET105_IsoTrk50", "HLT_MET120_IsoTrk50", "HLT_MediumChargedIsoPFTau180HighPtRelaxedIso_Trk50_eta2p1", "HLT_MediumChargedIsoPFTau180HighPtRelaxedIso_Trk50_eta2p1_1pr", "HLT_MediumChargedIsoPFTau50_Trk30_eta2p1_1pr", "HLT_MediumChargedIsoPFTau50_Trk30_eta2p1_1pr_MET100", "HLT_MediumChargedIsoPFTau50_Trk30_eta2p1_1pr_MET110", "HLT_MediumChargedIsoPFTau50_Trk30_eta2p1_1pr_MET120", "HLT_MediumChargedIsoPFTau50_Trk30_eta2p1_1pr_MET130", "HLT_MediumChargedIsoPFTau50_Trk30_eta2p1_1pr_MET90", "HLT_MonoCentralPFJet80_PFMETNoMu110_PFMHTNoMu110_IDTight", "HLT_MonoCentralPFJet80_PFMETNoMu120_PFMHTNoMu120_IDTight", "HLT_MonoCentralPFJet80_PFMETNoMu130_PFMHTNoMu130_IDTight", "HLT_MonoCentralPFJet80_PFMETNoMu140_PFMHTNoMu140_IDTight", "HLT_Mu10_TrkIsoVVL_DiPFJet40_DEta3p5_MJJ750_HTT350_PFMETNoMu60", "HLT_Mu12_DoublePFJets100_CaloBTagCSV_p33", "HLT_Mu12_DoublePFJets200_CaloBTagCSV_p33", "HLT_Mu12_DoublePFJets350_CaloBTagCSV_p33", "HLT_Mu12_DoublePFJets40MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_Mu12_DoublePFJets40_CaloBTagCSV_p33", "HLT_Mu12_DoublePFJets54MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_Mu12_DoublePFJets62MaxDeta1p6_DoubleCaloBTagCSV_p33", "HLT_Mu12_DoublePhoton20", "HLT_Mu12_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL", "HLT_Mu12_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ", "HLT_Mu15_IsoVVVL_PFHT450", "HLT_Mu15_IsoVVVL_PFHT450_CaloBTagCSV_4p5", "HLT_Mu15_IsoVVVL_PFHT450_PFMET50", "HLT_Mu15_IsoVVVL_PFHT600", "HLT_Mu17", "HLT_Mu17_Photon30_IsoCaloId", "HLT_Mu17_TrkIsoVVL", "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL", "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ", "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8", "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass8", "HLT_Mu18_Mu9", "HLT_Mu18_Mu9_DZ", "HLT_Mu18_Mu9_SameSign", "HLT_Mu18_Mu9_SameSign_DZ", "HLT_Mu19", "HLT_Mu19_TrkIsoVVL", "HLT_Mu19_TrkIsoVVL_Mu9_TrkIsoVVL", "HLT_Mu19_TrkIsoVVL_Mu9_TrkIsoVVL_DZ", "HLT_Mu19_TrkIsoVVL_Mu9_TrkIsoVVL_DZ_Mass3p8", "HLT_Mu19_TrkIsoVVL_Mu9_TrkIsoVVL_DZ_Mass8", "HLT_Mu20", "HLT_Mu20_Mu10", "HLT_Mu20_Mu10_DZ", "HLT_Mu20_Mu10_SameSign", "HLT_Mu20_Mu10_SameSign_DZ", "HLT_Mu20_TkMu0_Phi", "HLT_Mu23_Mu12", "HLT_Mu23_Mu12_DZ", "HLT_Mu23_Mu12_SameSign", "HLT_Mu23_Mu12_SameSign_DZ", "HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL", "HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL_DZ", "HLT_Mu25_TkMu0_Onia", "HLT_Mu25_TkMu0_Phi", "HLT_Mu27", "HLT_Mu27_Ele37_CaloIdL_MW", "HLT_Mu30_TkMu0_Onia", "HLT_Mu37_Ele27_CaloIdL_MW", "HLT_Mu37_TkMu27", "HLT_Mu3_PFJet40", "HLT_Mu43NoFiltersNoVtx_Photon43_CaloIdL", "HLT_Mu48NoFiltersNoVtx_Photon48_CaloIdL", "HLT_Mu50", "HLT_Mu50_IsoVVVL_PFHT450", "HLT_Mu55", "HLT_Mu7p5_L2Mu2_Jpsi", "HLT_Mu7p5_L2Mu2_Upsilon", "HLT_Mu7p5_Track2_Jpsi", "HLT_Mu7p5_Track2_Upsilon", "HLT_Mu7p5_Track3p5_Jpsi", "HLT_Mu7p5_Track3p5_Upsilon", "HLT_Mu7p5_Track7_Jpsi", "HLT_Mu7p5_Track7_Upsilon", "HLT_Mu8", "HLT_Mu8_DiEle12_CaloIdL_TrackIdL", "HLT_Mu8_DiEle12_CaloIdL_TrackIdL_DZ", "HLT_Mu8_Ele8_CaloIdM_TrackIdM_Mass8_PFHT350", "HLT_Mu8_Ele8_CaloIdM_TrackIdM_Mass8_PFHT350_DZ", "HLT_Mu8_TrkIsoVVL", "HLT_Mu8_TrkIsoVVL_DiPFJet40_DEta3p5_MJJ750_HTT300_PFMETNoMu60", "HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL", "HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ", "HLT_OldMu100", "HLT_PFHT1050", "HLT_PFHT180", "HLT_PFHT250", "HLT_PFHT300PT30_QuadPFJet_75_60_45_40", "HLT_PFHT300PT30_QuadPFJet_75_60_45_40_TriplePFBTagCSV_3p0", "HLT_PFHT350", "HLT_PFHT350MinPFJet15", "HLT_PFHT370", "HLT_PFHT380_SixPFJet32", "HLT_PFHT380_SixPFJet32_DoublePFBTagCSV_2p2", "HLT_PFHT380_SixPFJet32_DoublePFBTagDeepCSV_2p2", "HLT_PFHT430", "HLT_PFHT430_SixPFJet40", "HLT_PFHT430_SixPFJet40_PFBTagCSV_1p5", "HLT_PFHT500_PFMET100_PFMHT100_IDTight", "HLT_PFHT500_PFMET110_PFMHT110_IDTight", "HLT_PFHT510", "HLT_PFHT590", "HLT_PFHT680", "HLT_PFHT700_PFMET85_PFMHT85_IDTight", "HLT_PFHT700_PFMET95_PFMHT95_IDTight", "HLT_PFHT780", "HLT_PFHT800_PFMET75_PFMHT75_IDTight", "HLT_PFHT800_PFMET85_PFMHT85_IDTight", "HLT_PFHT890", "HLT_PFJet140", "HLT_PFJet200", "HLT_PFJet260", "HLT_PFJet320", "HLT_PFJet40", "HLT_PFJet400", "HLT_PFJet450", "HLT_PFJet500", "HLT_PFJet550", "HLT_PFJet60", "HLT_PFJet80", "HLT_PFJetFwd140", "HLT_PFJetFwd200", "HLT_PFJetFwd260", "HLT_PFJetFwd320", "HLT_PFJetFwd40", "HLT_PFJetFwd400", "HLT_PFJetFwd450", "HLT_PFJetFwd500", "HLT_PFJetFwd60", "HLT_PFJetFwd80", "HLT_PFMET100_PFMHT100_IDTight_CaloBTagCSV_3p1", "HLT_PFMET100_PFMHT100_IDTight_PFHT60", "HLT_PFMET110_PFMHT110_IDTight", "HLT_PFMET110_PFMHT110_IDTight_CaloBTagCSV_3p1", "HLT_PFMET120_PFMHT120_IDTight", "HLT_PFMET120_PFMHT120_IDTight_CaloBTagCSV_3p1", "HLT_PFMET120_PFMHT120_IDTight_PFHT60", "HLT_PFMET130_PFMHT130_IDTight", "HLT_PFMET130_PFMHT130_IDTight_CaloBTagCSV_3p1", "HLT_PFMET140_PFMHT140_IDTight", "HLT_PFMET140_PFMHT140_IDTight_CaloBTagCSV_3p1", "HLT_PFMET200_HBHECleaned", "HLT_PFMET200_HBHE_BeamHaloCleaned", "HLT_PFMET200_NotCleaned", "HLT_PFMET250_HBHECleaned", "HLT_PFMET300_HBHECleaned", "HLT_PFMETNoMu100_PFMHTNoMu100_IDTight_PFHT60", "HLT_PFMETNoMu110_PFMHTNoMu110_IDTight", "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight", "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60", "HLT_PFMETNoMu130_PFMHTNoMu130_IDTight", "HLT_PFMETNoMu140_PFMHTNoMu140_IDTight", "HLT_PFMETTypeOne100_PFMHT100_IDTight_PFHT60", "HLT_PFMETTypeOne110_PFMHT110_IDTight", "HLT_PFMETTypeOne120_PFMHT120_IDTight", "HLT_PFMETTypeOne120_PFMHT120_IDTight_PFHT60", "HLT_PFMETTypeOne130_PFMHT130_IDTight", "HLT_PFMETTypeOne140_PFMHT140_IDTight", "HLT_PFMETTypeOne200_HBHE_BeamHaloCleaned", "HLT_Photon120", "HLT_Photon120_R9Id90_HE10_IsoM", "HLT_Photon150", "HLT_Photon165_R9Id90_HE10_IsoM", "HLT_Photon175", "HLT_Photon200", "HLT_Photon20_HoverELoose", "HLT_Photon25", "HLT_Photon300_NoHE", "HLT_Photon30_HoverELoose", "HLT_Photon33", "HLT_Photon40_HoverELoose", "HLT_Photon50", "HLT_Photon50_HoverELoose", "HLT_Photon50_R9Id90_HE10_IsoM", "HLT_Photon50_R9Id90_HE10_IsoM_EBOnly_PFJetsMJJ300DEta3_PFMET50", "HLT_Photon60_HoverELoose", "HLT_Photon60_R9Id90_CaloIdL_IsoL", "HLT_Photon60_R9Id90_CaloIdL_IsoL_DisplacedIdL", "HLT_Photon60_R9Id90_CaloIdL_IsoL_DisplacedIdL_PFHT350MinPFJet15", "HLT_Photon75", "HLT_Photon75_R9Id90_HE10_IsoM", "HLT_Photon75_R9Id90_HE10_IsoM_EBOnly_PFJetsMJJ300DEta3", "HLT_Photon75_R9Id90_HE10_IsoM_EBOnly_PFJetsMJJ600DEta3", "HLT_Photon90", "HLT_Photon90_CaloIdL_PFHT700", "HLT_Photon90_R9Id90_HE10_IsoM", "HLT_Physics", "HLT_Physics_part0", "HLT_Physics_part1", "HLT_Physics_part2", "HLT_Physics_part3", "HLT_Physics_part4", "HLT_Physics_part5", "HLT_Physics_part6", "HLT_Physics_part7", "HLT_QuadPFJet103_88_75_15", "HLT_QuadPFJet103_88_75_15_BTagCSV_p013_VBF2", "HLT_QuadPFJet103_88_75_15_DoubleBTagCSV_p013_p08_VBF1", "HLT_QuadPFJet105_88_76_15", "HLT_QuadPFJet105_88_76_15_BTagCSV_p013_VBF2", "HLT_QuadPFJet105_90_76_15_DoubleBTagCSV_p013_p08_VBF1", "HLT_QuadPFJet111_90_80_15", "HLT_QuadPFJet111_90_80_15_BTagCSV_p013_VBF2", "HLT_QuadPFJet111_90_80_15_DoubleBTagCSV_p013_p08_VBF1", "HLT_QuadPFJet98_83_71_15", "HLT_QuadPFJet98_83_71_15_BTagCSV_p013_VBF2", "HLT_QuadPFJet98_83_71_15_DoubleBTagCSV_p013_p08_VBF1", "HLT_Random", "HLT_Rsq0p35", "HLT_Rsq0p40", "HLT_RsqMR300_Rsq0p09_MR200", "HLT_RsqMR300_Rsq0p09_MR200_4jet", "HLT_RsqMR320_Rsq0p09_MR200", "HLT_RsqMR320_Rsq0p09_MR200_4jet", "HLT_SingleJet30_Mu12_SinglePFJet40", "HLT_Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15", "HLT_Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15_Charge1", "HLT_Tau3Mu_Mu7_Mu1_TkMu1_Tau15", "HLT_Tau3Mu_Mu7_Mu1_TkMu1_Tau15_Charge1", "HLT_TkMu100", "HLT_Trimuon5_3p5_2_Upsilon_Muon", "HLT_TripleJet110_35_35_Mjj650_PFMET110", "HLT_TripleJet110_35_35_Mjj650_PFMET120", "HLT_TripleJet110_35_35_Mjj650_PFMET130", "HLT_TripleMu_10_5_5_DZ", "HLT_TripleMu_12_10_5", "HLT_TripleMu_5_3_3_Mass3p8to60_DCA", "HLT_TripleMu_5_3_3_Mass3p8to60_DZ", "HLT_TriplePhoton_20_20_20_CaloIdLV2", "HLT_TriplePhoton_20_20_20_CaloIdLV2_R9IdVL", "HLT_TriplePhoton_30_30_10_CaloIdLV2", "HLT_TriplePhoton_30_30_10_CaloIdLV2_R9IdVL", "HLT_TriplePhoton_35_35_5_CaloIdLV2_R9IdVL", "HLT_TrkMu12_DoubleTrkMu5NoFiltersNoVtx", "HLT_TrkMu16_DoubleTrkMu6NoFiltersNoVtx", "HLT_TrkMu17_DoubleTrkMu8NoFiltersNoVtx", "HLT_UncorrectedJetE30_NoBPTX", "HLT_UncorrectedJetE30_NoBPTX3BX", "HLT_UncorrectedJetE60_NoBPTX3BX", "HLT_UncorrectedJetE70_NoBPTX3BX", "HLT_VBF_DoubleLooseChargedIsoPFTau20_Trk1_eta2p1_Reg", "HLT_VBF_DoubleMediumChargedIsoPFTau20_Trk1_eta2p1_Reg", "HLT_VBF_DoubleTightChargedIsoPFTau20_Trk1_eta2p1_Reg", "HLT_ZeroBias", "HLT_ZeroBias_FirstBXAfterTrain", "HLT_ZeroBias_FirstCollisionAfterAbortGap", "HLT_ZeroBias_FirstCollisionInTrain", "HLT_ZeroBias_IsolatedBunches", "HLT_ZeroBias_LastCollisionInTrain", "HLT_ZeroBias_part0", "HLT_ZeroBias_part1", "HLT_ZeroBias_part2", "HLT_ZeroBias_part3", "HLT_ZeroBias_part4", "HLT_ZeroBias_part5", "HLT_ZeroBias_part6", "HLT_ZeroBias_part7", "HLTriggerFinalPath", "HLTriggerFirstPath", "HTXS_Higgs_pt", "HTXS_Higgs_y", "HTXS_njets25", "HTXS_njets30", "HTXS_stage1_1_cat_pTjet25GeV", "HTXS_stage1_1_cat_pTjet30GeV", "HTXS_stage1_1_fine_cat_pTjet25GeV", "HTXS_stage1_1_fine_cat_pTjet30GeV", "HTXS_stage1_2_cat_pTjet25GeV", "HTXS_stage1_2_cat_pTjet30GeV", "HTXS_stage1_2_fine_cat_pTjet25GeV", "HTXS_stage1_2_fine_cat_pTjet30GeV", "HTXS_stage_0", "HTXS_stage_1_pTjet25", "HTXS_stage_1_pTjet30", "IsoTrack_charge", "IsoTrack_dxy", "IsoTrack_dz", "IsoTrack_eta", "IsoTrack_fromPV", "IsoTrack_isFromLostTrack", "IsoTrack_isHighPurityTrack", "IsoTrack_isPFcand", "IsoTrack_miniPFRelIso_all", "IsoTrack_miniPFRelIso_chg", "IsoTrack_pdgId", "IsoTrack_pfRelIso03_all", "IsoTrack_pfRelIso03_chg", "IsoTrack_phi", "IsoTrack_pt", "Jet_area", "Jet_bRegCorr", "Jet_bRegRes", "Jet_btagCSVV2", "Jet_btagDeepB", "Jet_btagDeepCvB", "Jet_btagDeepCvL", "Jet_btagDeepFlavB", "Jet_btagDeepFlavCvB", "Jet_btagDeepFlavCvL", "Jet_btagDeepFlavQG", "Jet_cRegCorr", "Jet_cRegRes", "Jet_chEmEF", "Jet_chFPV0EF", "Jet_chHEF", "Jet_cleanmask", "Jet_electronIdx1", "Jet_electronIdx2", "Jet_eta", "Jet_genJetIdx", "Jet_hadronFlavour", "Jet_hfadjacentEtaStripsSize", "Jet_hfcentralEtaStripSize", "Jet_hfsigmaEtaEta", "Jet_hfsigmaPhiPhi", "Jet_jetId", "Jet_mass", "Jet_muEF", "Jet_muonIdx1", "Jet_muonIdx2", "Jet_muonSubtrFactor", "Jet_nConstituents", "Jet_nElectrons", "Jet_nMuons", "Jet_neEmEF", "Jet_neHEF", "Jet_partonFlavour", "Jet_phi", "Jet_pt", "Jet_puId", "Jet_puIdDisc", "Jet_qgl", "Jet_rawFactor", "L1PreFiringWeight_Dn", "L1PreFiringWeight_ECAL_Dn", "L1PreFiringWeight_ECAL_Nom", "L1PreFiringWeight_ECAL_Up", "L1PreFiringWeight_Muon_Nom", "L1PreFiringWeight_Muon_StatDn", "L1PreFiringWeight_Muon_StatUp", "L1PreFiringWeight_Muon_SystDn", "L1PreFiringWeight_Muon_SystUp", "L1PreFiringWeight_Nom", "L1PreFiringWeight_Up", "L1Reco_step", "L1_AlwaysTrue", "L1_BPTX_AND_Ref1_VME", "L1_BPTX_AND_Ref3_VME", "L1_BPTX_AND_Ref4_VME", "L1_BPTX_BeamGas_B1_VME", "L1_BPTX_BeamGas_B2_VME", "L1_BPTX_BeamGas_Ref1_VME", "L1_BPTX_BeamGas_Ref2_VME", "L1_BPTX_NotOR_VME", "L1_BPTX_OR_Ref3_VME", "L1_BPTX_OR_Ref4_VME", "L1_BPTX_RefAND_VME", "L1_BptxMinus", "L1_BptxOR", "L1_BptxPlus", "L1_BptxXOR", "L1_CDC_SingleMu_3_er1p2_TOP120_DPHI2p618_3p142", "L1_DoubleEG6_HTT240er", "L1_DoubleEG6_HTT250er", "L1_DoubleEG6_HTT255er", "L1_DoubleEG6_HTT270er", "L1_DoubleEG6_HTT300er", "L1_DoubleEG8er2p6_HTT255er", "L1_DoubleEG8er2p6_HTT270er", "L1_DoubleEG8er2p6_HTT300er", "L1_DoubleEG_15_10", "L1_DoubleEG_18_17", "L1_DoubleEG_20_18", "L1_DoubleEG_22_10", "L1_DoubleEG_22_12", "L1_DoubleEG_22_15", "L1_DoubleEG_23_10", "L1_DoubleEG_24_17", "L1_DoubleEG_25_12", "L1_DoubleEG_25_13", "L1_DoubleEG_25_14", "L1_DoubleEG_LooseIso23_10", "L1_DoubleEG_LooseIso24_10", "L1_DoubleIsoTau28er2p1", "L1_DoubleIsoTau30er2p1", "L1_DoubleIsoTau32er2p1", "L1_DoubleIsoTau33er2p1", "L1_DoubleIsoTau34er2p1", "L1_DoubleIsoTau35er2p1", "L1_DoubleIsoTau36er2p1", "L1_DoubleIsoTau38er2p1", "L1_DoubleJet100er2p3_dEta_Max1p6", "L1_DoubleJet100er2p7", "L1_DoubleJet112er2p3_dEta_Max1p6", "L1_DoubleJet112er2p7", "L1_DoubleJet120er2p7", "L1_DoubleJet150er2p7", "L1_DoubleJet30_Mass_Min300_dEta_Max1p5", "L1_DoubleJet30_Mass_Min320_dEta_Max1p5", "L1_DoubleJet30_Mass_Min340_dEta_Max1p5", "L1_DoubleJet30_Mass_Min360_dEta_Max1p5", "L1_DoubleJet30_Mass_Min380_dEta_Max1p5", "L1_DoubleJet30_Mass_Min400_Mu10", "L1_DoubleJet30_Mass_Min400_Mu6", "L1_DoubleJet30_Mass_Min400_dEta_Max1p5", "L1_DoubleJet35_rmovlp_IsoTau45_Mass_Min450", "L1_DoubleJet40er2p7", "L1_DoubleJet50er2p7", "L1_DoubleJet60er2p7", "L1_DoubleJet60er2p7_ETM100", "L1_DoubleJet60er2p7_ETM60", "L1_DoubleJet60er2p7_ETM70", "L1_DoubleJet60er2p7_ETM80", "L1_DoubleJet60er2p7_ETM90", "L1_DoubleJet80er2p7", "L1_DoubleJet_100_30_DoubleJet30_Mass_Min620", "L1_DoubleJet_100_35_DoubleJet35_Mass_Min620", "L1_DoubleJet_110_35_DoubleJet35_Mass_Min620", "L1_DoubleJet_110_40_DoubleJet40_Mass_Min620", "L1_DoubleJet_115_35_DoubleJet35_Mass_Min620", "L1_DoubleJet_115_40_DoubleJet40_Mass_Min620", "L1_DoubleJet_90_30_DoubleJet30_Mass_Min620", "L1_DoubleLooseIsoEG22er2p1", "L1_DoubleLooseIsoEG24er2p1", "L1_DoubleMu0", "L1_DoubleMu0_ETM40", "L1_DoubleMu0_ETM55", "L1_DoubleMu0_ETM60", "L1_DoubleMu0_ETM65", "L1_DoubleMu0_ETM70", "L1_DoubleMu0_SQ", "L1_DoubleMu0_SQ_OS", "L1_DoubleMu0er1p4_SQ_OS_dR_Max1p4", "L1_DoubleMu0er1p4_dEta_Max1p8_OS", "L1_DoubleMu0er1p5_SQ_OS", "L1_DoubleMu0er1p5_SQ_OS_dR_Max1p4", "L1_DoubleMu0er1p5_SQ_dR_Max1p4", "L1_DoubleMu0er2_SQ_dR_Max1p4", "L1_DoubleMu18er2p1", "L1_DoubleMu22er2p1", "L1_DoubleMu3_OS_DoubleEG7p5Upsilon", "L1_DoubleMu3_SQ_ETMHF40_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_ETMHF50_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_ETMHF60_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_ETMHF70_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_ETMHF80_Jet60_OR_DoubleJet30", "L1_DoubleMu3_SQ_HTT100er", "L1_DoubleMu3_SQ_HTT200er", "L1_DoubleMu3_SQ_HTT220er", "L1_DoubleMu3_SQ_HTT240er", "L1_DoubleMu4_OS_EG12", "L1_DoubleMu4_SQ_OS", "L1_DoubleMu4_SQ_OS_dR_Max1p2", "L1_DoubleMu4p5_SQ", "L1_DoubleMu4p5_SQ_OS", "L1_DoubleMu4p5_SQ_OS_dR_Max1p2", "L1_DoubleMu4p5er2p0_SQ_OS", "L1_DoubleMu4p5er2p0_SQ_OS_Mass7to18", "L1_DoubleMu5Upsilon_OS_DoubleEG3", "L1_DoubleMu5_OS_EG12", "L1_DoubleMu5_SQ_OS", "L1_DoubleMu5_SQ_OS_Mass7to18", "L1_DoubleMu6_SQ_OS", "L1_DoubleMu7_EG7", "L1_DoubleMu7_SQ_EG7", "L1_DoubleMu8_SQ", "L1_DoubleMu_10_0_dEta_Max1p8", "L1_DoubleMu_11_4", "L1_DoubleMu_12_5", "L1_DoubleMu_12_8", "L1_DoubleMu_13_6", "L1_DoubleMu_15_5", "L1_DoubleMu_15_5_SQ", "L1_DoubleMu_15_7", "L1_DoubleMu_15_7_SQ", "L1_DoubleMu_15_7_SQ_Mass_Min4", "L1_DoubleMu_20_2_SQ_Mass_Max20", "L1_DoubleTau50er2p1", "L1_DoubleTau70er2p1", "L1_EG25er2p1_HTT125er", "L1_EG27er2p1_HTT200er", "L1_ETM100", "L1_ETM100_Jet60_dPhi_Min0p4", "L1_ETM105", "L1_ETM110", "L1_ETM110_Jet60_dPhi_Min0p4", "L1_ETM115", "L1_ETM120", "L1_ETM150", "L1_ETM30", "L1_ETM40", "L1_ETM50", "L1_ETM60", "L1_ETM70", "L1_ETM75", "L1_ETM75_Jet60_dPhi_Min0p4", "L1_ETM80", "L1_ETM80_Jet60_dPhi_Min0p4", "L1_ETM85", "L1_ETM90", "L1_ETM90_Jet60_dPhi_Min0p4", "L1_ETM95", "L1_ETMHF100", "L1_ETMHF100_HTT60er", "L1_ETMHF100_Jet60_OR_DiJet30woTT28", "L1_ETMHF100_Jet60_OR_DoubleJet30", "L1_ETMHF100_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETMHF110", "L1_ETMHF110_HTT60er", "L1_ETMHF110_Jet60_OR_DiJet30woTT28", "L1_ETMHF110_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETMHF120", "L1_ETMHF120_HTT60er", "L1_ETMHF120_Jet60_OR_DiJet30woTT28", "L1_ETMHF150", "L1_ETMHF70", "L1_ETMHF70_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETMHF80", "L1_ETMHF80_HTT60er", "L1_ETMHF80_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETMHF90", "L1_ETMHF90_HTT60er", "L1_ETMHF90_Jet90_OR_DoubleJet45_OR_TripleJet30", "L1_ETT100_BptxAND", "L1_ETT110_BptxAND", "L1_ETT40_BptxAND", "L1_ETT50_BptxAND", "L1_ETT60_BptxAND", "L1_ETT70_BptxAND", "L1_ETT75_BptxAND", "L1_ETT80_BptxAND", "L1_ETT85_BptxAND", "L1_ETT90_BptxAND", "L1_ETT95_BptxAND", "L1_FirstBunchAfterTrain", "L1_FirstBunchInTrain", "L1_FirstCollisionInOrbit", "L1_FirstCollisionInTrain", "L1_HTT120er", "L1_HTT160er", "L1_HTT200er", "L1_HTT220er", "L1_HTT240er", "L1_HTT250er_QuadJet_70_55_40_35_er2p5", "L1_HTT255er", "L1_HTT270er", "L1_HTT280er", "L1_HTT280er_QuadJet_70_55_40_35_er2p5", "L1_HTT300er", "L1_HTT300er_QuadJet_70_55_40_35_er2p5", "L1_HTT320er", "L1_HTT320er_QuadJet_70_55_40_40_er2p4", "L1_HTT320er_QuadJet_70_55_40_40_er2p5", "L1_HTT320er_QuadJet_70_55_45_45_er2p5", "L1_HTT340er", "L1_HTT340er_QuadJet_70_55_40_40_er2p5", "L1_HTT340er_QuadJet_70_55_45_45_er2p5", "L1_HTT380er", "L1_HTT400er", "L1_HTT450er", "L1_HTT500er", "L1_IsoEG33_Mt40", "L1_IsoEG33_Mt44", "L1_IsoEG33_Mt48", "L1_IsoTau40er_ETM100", "L1_IsoTau40er_ETM105", "L1_IsoTau40er_ETM110", "L1_IsoTau40er_ETM115", "L1_IsoTau40er_ETM120", "L1_IsoTau40er_ETM80", "L1_IsoTau40er_ETM85", "L1_IsoTau40er_ETM90", "L1_IsoTau40er_ETM95", "L1_IsoTau40er_ETMHF100", "L1_IsoTau40er_ETMHF110", "L1_IsoTau40er_ETMHF120", "L1_IsoTau40er_ETMHF80", "L1_IsoTau40er_ETMHF90", "L1_IsolatedBunch", "L1_LastCollisionInTrain", "L1_LooseIsoEG22er2p1_IsoTau26er2p1_dR_Min0p3", "L1_LooseIsoEG24er2p1_HTT100er", "L1_LooseIsoEG24er2p1_IsoTau27er2p1_dR_Min0p3", "L1_LooseIsoEG24er2p1_Jet26er2p7_dR_Min0p3", "L1_LooseIsoEG24er2p1_TripleJet_26er2p7_26_26er2p7", "L1_LooseIsoEG26er2p1_HTT100er", "L1_LooseIsoEG26er2p1_Jet34er2p7_dR_Min0p3", "L1_LooseIsoEG28er2p1_HTT100er", "L1_LooseIsoEG28er2p1_Jet34er2p7_dR_Min0p3", "L1_LooseIsoEG30er2p1_Jet34er2p7_dR_Min0p3", "L1_MU20_EG15", "L1_MinimumBiasHF0_AND_BptxAND", "L1_MinimumBiasHF0_OR_BptxAND", "L1_Mu10er2p1_ETM30", "L1_Mu10er2p3_Jet32er2p3_dR_Max0p4_DoubleJet32er2p3_dEta_Max1p6", "L1_Mu12_EG10", "L1_Mu12er2p3_Jet40er2p3_dR_Max0p4_DoubleJet40er2p3_dEta_Max1p6", "L1_Mu14er2p1_ETM30", "L1_Mu15_HTT100er", "L1_Mu18_HTT100er", "L1_Mu18_Jet24er2p7", "L1_Mu18er2p1_IsoTau26er2p1", "L1_Mu18er2p1_Tau24er2p1", "L1_Mu20_EG10", "L1_Mu20_EG17", "L1_Mu20_LooseIsoEG6", "L1_Mu20er2p1_IsoTau26er2p1", "L1_Mu20er2p1_IsoTau27er2p1", "L1_Mu22er2p1_IsoTau28er2p1", "L1_Mu22er2p1_IsoTau30er2p1", "L1_Mu22er2p1_IsoTau32er2p1", "L1_Mu22er2p1_IsoTau33er2p1", "L1_Mu22er2p1_IsoTau34er2p1", "L1_Mu22er2p1_IsoTau35er2p1", "L1_Mu22er2p1_IsoTau36er2p1", "L1_Mu22er2p1_IsoTau38er2p1", "L1_Mu22er2p1_IsoTau40er2p1", "L1_Mu22er2p1_Tau50er2p1", "L1_Mu22er2p1_Tau70er2p1", "L1_Mu23_EG10", "L1_Mu23_LooseIsoEG10", "L1_Mu3_Jet120er2p7_dEta_Max0p4_dPhi_Max0p4", "L1_Mu3_Jet16er2p7_dEta_Max0p4_dPhi_Max0p4", "L1_Mu3_Jet30er2p5", "L1_Mu3_Jet60er2p7_dEta_Max0p4_dPhi_Max0p4", "L1_Mu5_EG15", "L1_Mu5_EG20", "L1_Mu5_EG23", "L1_Mu5_LooseIsoEG18", "L1_Mu5_LooseIsoEG20", "L1_Mu6_DoubleEG10", "L1_Mu6_DoubleEG17", "L1_Mu6_HTT200er", "L1_Mu6_HTT240er", "L1_Mu6_HTT250er", "L1_Mu7_EG23", "L1_Mu7_LooseIsoEG20", "L1_Mu7_LooseIsoEG23", "L1_Mu8_HTT150er", "L1_NotBptxOR", "L1_QuadJet36er2p7_IsoTau52er2p1", "L1_QuadJet36er2p7_Tau52", "L1_QuadJet40er2p7", "L1_QuadJet50er2p7", "L1_QuadJet60er2p7", "L1_QuadMu0", "L1_SingleEG10", "L1_SingleEG15", "L1_SingleEG18", "L1_SingleEG24", "L1_SingleEG26", "L1_SingleEG28", "L1_SingleEG2_BptxAND", "L1_SingleEG30", "L1_SingleEG32", "L1_SingleEG34", "L1_SingleEG34er2p1", "L1_SingleEG36", "L1_SingleEG36er2p1", "L1_SingleEG38", "L1_SingleEG38er2p1", "L1_SingleEG40", "L1_SingleEG42", "L1_SingleEG45", "L1_SingleEG5", "L1_SingleEG50", "L1_SingleIsoEG18", "L1_SingleIsoEG18er2p1", "L1_SingleIsoEG20", "L1_SingleIsoEG20er2p1", "L1_SingleIsoEG22", "L1_SingleIsoEG22er2p1", "L1_SingleIsoEG24", "L1_SingleIsoEG24er2p1", "L1_SingleIsoEG26", "L1_SingleIsoEG26er2p1", "L1_SingleIsoEG28", "L1_SingleIsoEG28er2p1", "L1_SingleIsoEG30", "L1_SingleIsoEG30er2p1", "L1_SingleIsoEG32", "L1_SingleIsoEG32er2p1", "L1_SingleIsoEG33er2p1", "L1_SingleIsoEG34", "L1_SingleIsoEG34er2p1", "L1_SingleIsoEG35", "L1_SingleIsoEG35er2p1", "L1_SingleIsoEG36", "L1_SingleIsoEG36er2p1", "L1_SingleIsoEG37", "L1_SingleIsoEG38", "L1_SingleIsoEG38er2p1", "L1_SingleIsoEG40", "L1_SingleIsoEG40er2p1", "L1_SingleJet120", "L1_SingleJet120_FWD", "L1_SingleJet12_BptxAND", "L1_SingleJet140", "L1_SingleJet150", "L1_SingleJet16", "L1_SingleJet160", "L1_SingleJet170", "L1_SingleJet180", "L1_SingleJet20", "L1_SingleJet200", "L1_SingleJet20er2p7_NotBptxOR", "L1_SingleJet20er2p7_NotBptxOR_3BX", "L1_SingleJet35", "L1_SingleJet35_FWD", "L1_SingleJet35_HFm", "L1_SingleJet35_HFp", "L1_SingleJet43er2p7_NotBptxOR_3BX", "L1_SingleJet46er2p7_NotBptxOR_3BX", "L1_SingleJet60", "L1_SingleJet60_FWD", "L1_SingleJet60_HFm", "L1_SingleJet60_HFp", "L1_SingleJet90", "L1_SingleJet90_FWD", "L1_SingleMu0_BMTF", "L1_SingleMu0_EMTF", "L1_SingleMu0_OMTF", "L1_SingleMu10_LowQ", "L1_SingleMu11_LowQ", "L1_SingleMu12_LowQ_BMTF", "L1_SingleMu12_LowQ_EMTF", "L1_SingleMu12_LowQ_OMTF", "L1_SingleMu14er2p1", "L1_SingleMu16", "L1_SingleMu16er2p1", "L1_SingleMu18", "L1_SingleMu18er2p1", "L1_SingleMu20", "L1_SingleMu20er2p1", "L1_SingleMu22", "L1_SingleMu22_BMTF", "L1_SingleMu22_EMTF", "L1_SingleMu22_OMTF", "L1_SingleMu22er2p1", "L1_SingleMu25", "L1_SingleMu3", "L1_SingleMu30", "L1_SingleMu5", "L1_SingleMu7", "L1_SingleMuCosmics", "L1_SingleMuCosmics_BMTF", "L1_SingleMuCosmics_EMTF", "L1_SingleMuCosmics_OMTF", "L1_SingleMuOpen", "L1_SingleMuOpen_NotBptxOR", "L1_SingleMuOpen_NotBptxOR_3BX", "L1_SingleTau100er2p1", "L1_SingleTau120er2p1", "L1_SingleTau130er2p1", "L1_SingleTau140er2p1", "L1_SingleTau20", "L1_SingleTau80er2p1", "L1_TripleEG_14_10_8", "L1_TripleEG_18_17_8", "L1_TripleEG_LooseIso20_10_5", "L1_TripleJet_100_85_72_VBF", "L1_TripleJet_105_85_76_VBF", "L1_TripleJet_84_68_48_VBF", "L1_TripleJet_88_72_56_VBF", "L1_TripleJet_92_76_64_VBF", "L1_TripleJet_98_83_71_VBF", "L1_TripleMu0", "L1_TripleMu0_OQ", "L1_TripleMu3", "L1_TripleMu3_SQ", "L1_TripleMu_4_4_4", "L1_TripleMu_5OQ_3p5OQ_2p5OQ_DoubleMu_5_2p5_OQ_OS_Mass_5to17", "L1_TripleMu_5OQ_3p5OQ_2p5OQ_DoubleMu_5_2p5_OQ_OS_Mass_8to14", "L1_TripleMu_5SQ_3SQ_0OQ", "L1_TripleMu_5SQ_3SQ_0OQ_DoubleMu_5_3_SQ_OS_Mass_Max9", "L1_TripleMu_5SQ_3SQ_0_DoubleMu_5_3_SQ_OS_Mass_Max9", "L1_TripleMu_5_0_0", "L1_TripleMu_5_3_3", "L1_TripleMu_5_3p5_2p5", "L1_TripleMu_5_3p5_2p5_DoubleMu_5_2p5_OS_Mass_5to17", "L1_TripleMu_5_4_2p5_DoubleMu_5_2p5_OS_Mass_5to17", "L1_TripleMu_5_5_3", "L1_UnpairedBunchBptxMinus", "L1_UnpairedBunchBptxPlus", "L1_UnprefireableEvent", "L1_ZeroBias", "L1_ZeroBias_copy", "L1simulation_step", "LHEPart_eta", "LHEPart_incomingpz", "LHEPart_mass", "LHEPart_pdgId", "LHEPart_phi", "LHEPart_pt", "LHEPart_spin", "LHEPart_status", "LHEPdfWeight", "LHEReweightingWeight", "LHEScaleWeight", "LHEWeight_originalXWGTUP", "LHE_AlphaS", "LHE_HT", "LHE_HTIncoming", "LHE_Nb", "LHE_Nc", "LHE_Nglu", "LHE_Njets", "LHE_NpLO", "LHE_NpNLO", "LHE_Nuds", "LHE_Vpt", "LowPtElectron_ID", "LowPtElectron_charge", "LowPtElectron_convVeto", "LowPtElectron_convVtxRadius", "LowPtElectron_convWP", "LowPtElectron_deltaEtaSC", "LowPtElectron_dxy", "LowPtElectron_dxyErr", "LowPtElectron_dz", "LowPtElectron_dzErr", "LowPtElectron_eInvMinusPInv", "LowPtElectron_embeddedID", "LowPtElectron_energyErr", "LowPtElectron_eta", "LowPtElectron_genPartFlav", "LowPtElectron_genPartIdx", "LowPtElectron_hoe", "LowPtElectron_lostHits", "LowPtElectron_mass", "LowPtElectron_miniPFRelIso_all", "LowPtElectron_miniPFRelIso_chg", "LowPtElectron_pdgId", "LowPtElectron_phi", "LowPtElectron_pt", "LowPtElectron_ptbiased", "LowPtElectron_r9", "LowPtElectron_scEtOverPt", "LowPtElectron_sieie", "LowPtElectron_unbiased", "MET_MetUnclustEnUpDeltaX", "MET_MetUnclustEnUpDeltaY", "MET_covXX", "MET_covXY", "MET_covYY", "MET_fiducialGenPhi", "MET_fiducialGenPt", "MET_phi", "MET_pt", "MET_significance", "MET_sumEt", "MET_sumPtUnclustered", "Muon_charge", "Muon_cleanmask", "Muon_dxy", "Muon_dxyErr", "Muon_dxybs", "Muon_dz", "Muon_dzErr", "Muon_eta", "Muon_fsrPhotonIdx", "Muon_genPartFlav", "Muon_genPartIdx", "Muon_highPtId", "Muon_highPurity", "Muon_inTimeMuon", "Muon_ip3d", "Muon_isGlobal", "Muon_isPFcand", "Muon_isStandalone", "Muon_isTracker", "Muon_jetIdx", "Muon_jetNDauCharged", "Muon_jetPtRelv2", "Muon_jetRelIso", "Muon_looseId", "Muon_mass", "Muon_mediumId", "Muon_mediumPromptId", "Muon_miniIsoId", "Muon_miniPFRelIso_all", "Muon_miniPFRelIso_chg", "Muon_multiIsoId", "Muon_mvaId", "Muon_mvaLowPt", "Muon_mvaLowPtId", "Muon_mvaTTH", "Muon_nStations", "Muon_nTrackerLayers", "Muon_pdgId", "Muon_pfIsoId", "Muon_pfRelIso03_all", "Muon_pfRelIso03_chg", "Muon_pfRelIso04_all", "Muon_phi", "Muon_pt", "Muon_ptErr", "Muon_puppiIsoId", "Muon_segmentComp", "Muon_sip3d", "Muon_softId", "Muon_softMva", "Muon_softMvaId", "Muon_tightCharge", "Muon_tightId", "Muon_tkIsoId", "Muon_tkRelIso", "Muon_triggerIdLoose", "Muon_tunepRelPt", "OtherPV_z", "PSWeight", "PV_chi2", "PV_ndof", "PV_npvs", "PV_npvsGood", "PV_score", "PV_x", "PV_y", "PV_z", "Photon_charge", "Photon_cleanmask", "Photon_cutBased", "Photon_cutBased_Fall17V1Bitmap", "Photon_dEscaleDown", "Photon_dEscaleUp", "Photon_dEsigmaDown", "Photon_dEsigmaUp", "Photon_eCorr", "Photon_electronIdx", "Photon_electronVeto", "Photon_energyErr", "Photon_eta", "Photon_genPartFlav", "Photon_genPartIdx", "Photon_hoe", "Photon_isScEtaEB", "Photon_isScEtaEE", "Photon_jetIdx", "Photon_mass", "Photon_mvaID", "Photon_mvaID_Fall17V1p1", "Photon_mvaID_WP80", "Photon_mvaID_WP90", "Photon_pdgId", "Photon_pfRelIso03_all", "Photon_pfRelIso03_chg", "Photon_phi", "Photon_pixelSeed", "Photon_pt", "Photon_r9", "Photon_seedGain", "Photon_sieie", "Photon_vidNestedWPBitmap", "Pileup_gpudensity", "Pileup_nPU", "Pileup_nTrueInt", "Pileup_pudensity", "Pileup_sumEOOT", "Pileup_sumLOOT", "PuppiMET_phi", "PuppiMET_phiJERDown", "PuppiMET_phiJERUp", "PuppiMET_phiJESDown", "PuppiMET_phiJESUp", "PuppiMET_phiUnclusteredDown", "PuppiMET_phiUnclusteredUp", "PuppiMET_pt", "PuppiMET_ptJERDown", "PuppiMET_ptJERUp", "PuppiMET_ptJESDown", "PuppiMET_ptJESUp", "PuppiMET_ptUnclusteredDown", "PuppiMET_ptUnclusteredUp", "PuppiMET_sumEt", "RawMET_phi", "RawMET_pt", "RawMET_sumEt", "RawPuppiMET_phi", "RawPuppiMET_pt", "RawPuppiMET_sumEt", "SV_charge", "SV_chi2", "SV_dlen", "SV_dlenSig", "SV_dxy", "SV_dxySig", "SV_eta", "SV_mass", "SV_ndof", "SV_ntracks", "SV_pAngle", "SV_phi", "SV_pt", "SV_x", "SV_y", "SV_z", "SoftActivityJetHT", "SoftActivityJetHT10", "SoftActivityJetHT2", "SoftActivityJetHT5", "SoftActivityJetNjets10", "SoftActivityJetNjets2", "SoftActivityJetNjets5", "SoftActivityJet_eta", "SoftActivityJet_phi", "SoftActivityJet_pt", "SubGenJetAK8_eta", "SubGenJetAK8_mass", "SubGenJetAK8_phi", "SubGenJetAK8_pt", "SubJet_btagCSVV2", "SubJet_btagDeepB", "SubJet_eta", "SubJet_hadronFlavour", "SubJet_mass", "SubJet_n2b1", "SubJet_n3b1", "SubJet_nBHadrons", "SubJet_nCHadrons", "SubJet_phi", "SubJet_pt", "SubJet_rawFactor", "SubJet_tau1", "SubJet_tau2", "SubJet_tau3", "SubJet_tau4", "Tau_charge", "Tau_chargedIso", "Tau_cleanmask", "Tau_decayMode", "Tau_dxy", "Tau_dz", "Tau_eta", "Tau_genPartFlav", "Tau_genPartIdx", "Tau_idAntiEleDeadECal", "Tau_idAntiMu", "Tau_idDecayModeOldDMs", "Tau_idDeepTau2017v2p1VSe", "Tau_idDeepTau2017v2p1VSjet", "Tau_idDeepTau2017v2p1VSmu", "Tau_jetIdx", "Tau_leadTkDeltaEta", "Tau_leadTkDeltaPhi", "Tau_leadTkPtOverTauPt", "Tau_mass", "Tau_neutralIso", "Tau_phi", "Tau_photonsOutsideSignalCone", "Tau_pt", "Tau_puCorr", "Tau_rawDeepTau2017v2p1VSe", "Tau_rawDeepTau2017v2p1VSjet", "Tau_rawDeepTau2017v2p1VSmu", "Tau_rawIso", "Tau_rawIsodR03", "TkMET_phi", "TkMET_pt", "TkMET_sumEt", "TrigObj_eta", "TrigObj_filterBits", "TrigObj_id", "TrigObj_l1charge", "TrigObj_l1iso", "TrigObj_l1pt", "TrigObj_l1pt_2", "TrigObj_l2pt", "TrigObj_phi", "TrigObj_pt", "boostedTau_charge", "boostedTau_chargedIso", "boostedTau_decayMode", "boostedTau_eta", "boostedTau_genPartFlav", "boostedTau_genPartIdx", "boostedTau_idAntiEle2018", "boostedTau_idAntiMu", "boostedTau_idMVAnewDM2017v2", "boostedTau_idMVAoldDM2017v2", "boostedTau_idMVAoldDMdR032017v2", "boostedTau_jetIdx", "boostedTau_leadTkDeltaEta", "boostedTau_leadTkDeltaPhi", "boostedTau_leadTkPtOverTauPt", "boostedTau_mass", "boostedTau_neutralIso", "boostedTau_phi", "boostedTau_photonsOutsideSignalCone", "boostedTau_pt", "boostedTau_puCorr", "boostedTau_rawAntiEle2018", "boostedTau_rawAntiEleCat2018", "boostedTau_rawIso", "boostedTau_rawIsodR03", "boostedTau_rawMVAnewDM2017v2", "boostedTau_rawMVAoldDM2017v2", "boostedTau_rawMVAoldDMdR032017v2", "btagWeight_CSVV2", "btagWeight_DeepCSVB", "event", "fixedGridRhoFastjetAll", "fixedGridRhoFastjetCentral", "fixedGridRhoFastjetCentralCalo", "fixedGridRhoFastjetCentralChargedPileUp", "fixedGridRhoFastjetCentralNeutral", "genTtbarId", "genWeight", "luminosityBlock", "nCorrT1METJet", "nElectron", "nFatJet", "nFsrPhoton", "nGenDressedLepton", "nGenIsolatedPhoton", "nGenJet", "nGenJetAK8", "nGenPart", "nGenVisTau", "nIsoTrack", "nJet", "nLHEPart", "nLHEPdfWeight", "nLHEReweightingWeight", "nLHEScaleWeight", "nLowPtElectron", "nMuon", "nOtherPV", "nPSWeight", "nPhoton", "nSV", "nSoftActivityJet", "nSubGenJetAK8", "nSubJet", "nTau", "nTrigObj", "nboostedTau", "run" ]
#for i in range(50):
#    df = RDataFrame("Events", chain[:firstNfilesinchain], npartitions=nPartitions, daskclient=client)
#    print(i)
#    sums = [df.Sum(column) for column in columns]
#    client.restart()
#    client.register_worker_plugin(UploadFile("./proxy"))
#    client.run(set_proxy)
#    sums[0].GetValue()

## Define preselection flow - MC

In [ ]:

%%time
def execute():
    if distributed == True:
        #df = RDataFrame("Events", chain[:firstNfilesinchain], npartitions=nPartitions, daskclient=client)
        df = RDataFrame("Events", chain, npartitions=nPartitions, daskclient=client)

    else:
        #chain = ["root://" + proxy + "//store/mc/RunIIFall17NanoAODv7/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/100000/5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root"]
        #chain = ["5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root"]
        df = RDataFrame("Events", chain[0])

    df_year = df.DefinePerSample("Year","GetYear(rdfslot_, rdfsampleinfo_)")
    df_sample = df_year.DefinePerSample("Sample", "GetSample(rdfslot_, rdfsampleinfo_)")

    nevents_df = {}
    gen_weight_df = {}
    #q2_weight = {}
    #ps_weight = {}
    #pdf_weight = {}
    df_samples = {}

    #### MCWeight_writer #### 
    for k,v in aggregated_samples.items():
        for sample in v:
            s = sample.label
            df_samples[s] = df_sample.Filter("Sample == {}".format(sampleDict[s]))
            nevents_df[s] = df_samples[s].Count()
            gen_weight_df[s] = df_samples[s].Sum("Generator_weight")
            #q2_weight[s] = [df_samples[s].Sum("LHEScaleWeight[{}]".format(j)) for j in range(0,7)]
            #ps_weight[s] = [df_samples[s].Sum("PSWeight[{}]".format(j)) for j in range(0,4)]
            #pdf_weight[s] = [df_samples[s].Sum("W_Pdf[{}]".format(j)) for j in range(0,15)]

    #### MET HLT Filter ####   #### working only for 2017 and 2018 (2016 requires a column that is not present in 2017 nd 2018, i.e. HLT_IsoTkMu24
    #df_METHLTFilter = df_sample.Define("METHLTFilter", "MET_HLT_Filter(Year, Flag_goodVertices, Flag_HBHENoiseFilter, Flag_HBHENoiseIsoFilter, Flag_EcalDeadCellTriggerPrimitiveFilter, Flag_BadPFMuonFilter, Flag_globalSuperTightHalo2016Filter, HLT_Ele27_WPTight_Gsf,  HLT_Ele32_WPTight_Gsf, HLT_IsoMu24, HLT_IsoMu27, HLT_Mu50, HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200)")\
    #                           .Filter("METHLTFilter == true", "MET HLT Filter")
    df_METHLTFilter = df_sample.Define("METHLTFilter", "MET_HLT_Filter_UL2017(Year, Flag_goodVertices, Flag_HBHENoiseFilter, Flag_HBHENoiseIsoFilter, Flag_EcalDeadCellTriggerPrimitiveFilter, Flag_BadPFMuonFilter, Flag_globalSuperTightHalo2016Filter, HLT_IsoMu27, HLT_Mu50, HLT_OldMu100, HLT_TkMu100,  HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200, Flag_ecalBadCalibFilter, Flag_BadPFMuonDzFilter, L1_SingleIsoEG30er2p1, L1_SingleIsoEG32, L1_SingleEG40, Flag_eeBadScFilter)")\
                               .Filter("METHLTFilter == true", "MET HLT Filter")
    #### preselection #####
    df_preselection = df_METHLTFilter.Filter("PV_ndof> 4 && abs(PV_z) < 24 && hypot(PV_x, PV_y)<2", "Good vertex")\
                                     .Define("HT_eventHT", "GetEventHT(Jet_pt, Jet_eta, Jet_phi, Jet_mass)")\
                                     .Define("PassMinReq", "Pass_min_req(Muon_looseId, Muon_pt, Muon_pfRelIso04_all, Muon_eta, Electron_mvaFall17V2Iso_WPL, Electron_jetRelIso, Electron_pt, Electron_eta, Tau_idDeepTau2017v2p1VSjet, Tau_idDeepTau2017v2p1VSe, Tau_idDeepTau2017v2p1VSmu, Tau_pt, Tau_eta, Jet_pt, Jet_eta, Jet_puId)")\
                                     .Filter("PassMinReq == true", "Minimum requests")

    #### lepSF #####
    df_LepSF = df_preselection.Define("ElectronSFs", "ElectronSFs(Electron_pt, Electron_eta, Electron_pdgId, Year)")\
                              .Define("Electron_effSF", "getFlattenedMatrixColumn(ElectronSFs, 3, 0)")\
                              .Define("Electron_effSF_errUp", "getFlattenedMatrixColumn(ElectronSFs, 3, 1)")\
                              .Define("Electron_effSF_errDown", "getFlattenedMatrixColumn(ElectronSFs, 3, 2)")\
                              .Define("MuonSFs", "MuonSFs(Muon_pt, Muon_eta, Muon_pdgId, Year)")\
                              .Define("Muon_effSF", "getFlattenedMatrixColumn(MuonSFs, 3, 0)")\
                              .Define("Muon_effSF_errUp", "getFlattenedMatrixColumn(MuonSFs, 3, 1)")\
                              .Define("Muon_effSF_errDown", "getFlattenedMatrixColumn(MuonSFs, 3, 2)")

    #### mht ####
    df_mht = df_LepSF.Define("MHT_pt_phi", "MHT_pt_phi(Electron_pt, Electron_eta, Electron_phi, Electron_mass, Electron_miniPFRelIso_all, Muon_pt, Muon_eta, Muon_phi, Muon_mass, Muon_miniPFRelIso_all, Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_muonIdx1, Jet_muonIdx2, Jet_electronIdx1, Jet_electronIdx2, nJet)")\
                     .Define("MHT_pt", "MHT_pt_phi[0]").Define("MHT_phi", "MHT_pt_phi[1]")\

    #### puWeight #####
    df_puWeight = df_mht.Define("puWeights", "puWeight(Year, Pileup_nTrueInt)")\
                        .Define("puWeight", "puWeights[0]")\
                        .Define("puWeightUp", "puWeights[1]")\
                        .Define("puWeightDown", "puWeights[2]")

    #### prefCorr ####
    df_prefCorr = df_puWeight.Define("prefCorrs","PrefCorr(Photon_pt, Photon_eta, Photon_jetIdx, Photon_electronIdx, Electron_pt, Electron_eta, Electron_jetIdx, Electron_photonIdx, Jet_pt, Jet_eta)")\
                             .Define("PrefireWeight", "prefCorrs[0]")\
                             .Define("PrefireWeightUp", "prefCorrs[1]")\
                             .Define("PrefireWeightDown", "prefCorrs[2]")

    #### ht ####
    df_ht = df_prefCorr.Define("HT_pt", "htProducer(nJet, Jet_pt)")


    #### btagSF ##### preselection part2
    df_btagSF = df_ht.Define("btagSFs",  'btagSF(Jet_pt, Jet_eta, Jet_hadronFlavour, Jet_btagDeepFlavB, Year, \"M\")')\
                     .Define("Jet_btagSF_deepjet_M", "getMatrixColumn(btagSFs, 0)")\
                     .Define("Jet_btagSF_deepjet_M_up", "getMatrixColumn(btagSFs, 1)")\
                     .Define("Jet_btagSF_deepjet_M_down", "getMatrixColumn(btagSFs, 2)")


    #### muonScaleRes ####
    df_muonScaleRes = df_btagSF.Define("muonCorrectedPTs", "muonScaleRes(Muon_pt, Muon_eta, Muon_phi, Muon_charge, Muon_nTrackerLayers, Muon_genPartIdx, GenPart_pt, Year)")\
                               .Define("Muon_corrected_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 0)")\
                               .Define("Muon_correctedUp_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 1)")\
                               .Define("Muon_correctedDown_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 2)")


    #### metCorrector and fatJetCorrector #### ####

    df_jme = df_muonScaleRes.Define("jetVars", "myJetVarCalc.produce(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_rawFactor, Jet_area, Jet_jetId, fixedGridRhoFastjetAll, (run<<20) + (luminosityBlock<<10) + event + 1 + int(Jet_eta[0]/.01), GenJet_pt, GenJet_eta, GenJet_phi, GenJet_mass)")\
                            .Define("Jet_pt_nom", "jetVars.pt(0)")\
                            .Define("Jet_mass_nom", "jetVars.mass(0)")\
                            .Define("fatjetVars", "myFatJetVarCalc.produce(FatJet_pt, FatJet_eta, FatJet_phi, FatJet_mass, FatJet_rawFactor, FatJet_area, FatJet_msoftdrop, FatJet_subJetIdx1, FatJet_subJetIdx2, SubJet_pt, SubJet_eta, SubJet_phi, SubJet_mass, {}, fixedGridRhoFastjetAll, (run<<20) + (luminosityBlock<<10) + event + 1 + int(Jet_eta[0]/.01) , GenJetAK8_pt, GenJetAK8_eta, GenJetAK8_phi, GenJetAK8_mass, SubGenJetAK8_pt, SubGenJetAK8_eta, SubGenJetAK8_phi, SubGenJetAK8_mass)")\
                            .Define("FatJet_pt_nom", "fatjetVars.pt(0)")\
                            .Define("FatJet_mass_nom", "fatjetVars.mass(0)")\
                            .Define("FatJet_msoftdrop_nom", "fatjetVars.msoftdrop(0)")\
                            .Define("metVars", "myType1METVarCalc.produce(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_rawFactor, Jet_area, Jet_muonSubtrFactor, Jet_neEmEF, Jet_chEmEF, Jet_jetId, fixedGridRhoFastjetAll, (run<<20) + (luminosityBlock<<10) + event + 1 + int(Jet_eta[0]/.01), GenJet_pt, GenJet_eta, GenJet_phi, GenJet_mass, RawMET_phi , RawMET_pt, MET_MetUnclustEnUpDeltaX, MET_MetUnclustEnUpDeltaY, CorrT1METJet_rawPt, CorrT1METJet_eta, CorrT1METJet_phi, CorrT1METJet_area, CorrT1METJet_muonSubtrFactor, {}, {})")\
                            .Define("MET_T1_pt_nom", "metVars.pt(0)")\
                            .Define("MET_T1_phi_nom", "metVars.phi(0)")\
                            .Define("metsVars", "myType1SmearMETVarCalc.produce(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_rawFactor, Jet_area, Jet_muonSubtrFactor, Jet_neEmEF, Jet_chEmEF, Jet_jetId, fixedGridRhoFastjetAll, (run<<20) + (luminosityBlock<<10) + event + 1 + int(Jet_eta[0]/.01), GenJet_pt, GenJet_eta, GenJet_phi, GenJet_mass, RawMET_phi , RawMET_pt, MET_MetUnclustEnUpDeltaX, MET_MetUnclustEnUpDeltaY, CorrT1METJet_rawPt, CorrT1METJet_eta, CorrT1METJet_phi, CorrT1METJet_area, CorrT1METJet_muonSubtrFactor, {}, {})")\
                            .Define("MET_T1Smear_pt_nom", "metsVars.pt(0)")\
                            .Define("MET_T1Smear_phi_nom", "metsVars.phi(0)")                   

    jetvariations = variations
    for n,v in enumerate(jetvariations[1:]):
        #print("Jet_pt_{}".format(v))
        if v in variations:
            #print("Jet_pt_{}".format(v))
            df_jme = df_jme.Define("Jet_pt_{}".format(v), "jetVars.pt({})".format(1+n)).Define("Jet_mass_{}".format(v), "jetVars.mass({})".format(1+n))
    for n,v in enumerate(fatjetvariations[1:]):
        if v in variations:
            df_jme = df_jme.Define("FatJet_pt_{}".format(v), "fatjetVars.pt({})".format(1+n)).Define("FatJet_mass_{}".format(v), "fatjetVars.mass({})".format(1+n)).Define("FatJet_msoftdrop_{}".format(v), "fatjetVars.msoftdrop({})".format(1+n))
    for n,v in enumerate(mett1variations[1:]):
        if v in variations:
            df_jme = df_jme.Define("MET_T1_pt_{}".format(v), "metVars.pt({})".format(1+n)).Define("MET_T1_phi_{}".format(v), "metVars.phi({})".format(1+n))  
    for n,v in enumerate(mett1smearvariations[1:]):
        if v in variations:
            df_jme = df_jme.Define("MET_T1Smear_pt_{}".format(v), "metsVars.pt({})".format(1+n)).Define("MET_T1Smear_phi_{}".format(v), "metsVars.phi({})".format(1+n))  

    ### book snapshot ####
    opts = ROOT.RDF.RSnapshotOptions()
    opts.fLazy = True

    if distributed == True:
        df_jme_lazy = df_jme.Snapshot("testTree", "./preselectionUL.root", branchlist, opts)
        #df_jme_lazy = df_jme.Histo1D("FatJet_pt_nom")
        #histos = {}
        #for b in branches:
        #    histos[b] = df_jme.Histo1D(b)
        #df_jme_lazy = df_jme.Count()
        #df_jme_lazy = df_jme.Sum("nJet")
        #df_jme_lazy = {}
        #branches = ['METHLTFilter', 'Electron_effSF', 'Muon_effSF', 'MHT_pt', 'puWeight', 'Jet_btagSF_deepjet_M', 'HT_pt', 'Muon_corrected_pt']
        #branches = ["Jet_pt_jesTotalup"]
        #print("branches:", branches) 
        #for b in branches:
        #    df_jme_lazy[b] = df_jme.Sum(b)
        #df_jme_lazy = dfs_jme[-1].Histo1D("HT_pt")

    else:
        #df_jme_lazy = df_jme.Snapshot("testTree", "testFile_MC_UL.root", branchlist, opts)
        df_jme_lazy = {}
        #branches = ['METHLTFilter', 'Electron_effSF', 'Muon_effSF', 'MHT_pt', 'puWeight', 'Jet_btagSF_deepjet_M', 'HT_pt', 'Muon_corrected_pt', "Jet_pt_jesTotaldown", "Jet_pt_jesTotalup", "Jet_pt"]
        for b in branches:
            df_jme_lazy[b] = df_jme.Sum(b)
        #df_jme_lazy = dfs_jme[-1].Snapshot("testTree", "testFile_MC.root", branchlist, opts)
    
    #print(df_jme_lazy[branches[-1]].GetValue())
    print(df_jme_lazy.GetValue())


In [ ]:

%%time
for i in range(0,1):
    print(i)
    client.restart()
    client.register_worker_plugin(UploadFile("./proxy"))
    client.run(set_proxy)
    execute()


In [ ]:
ERROR GENERATOR

In [ ]:
%%time
#ROOT.RDF.SaveGraph(df, "./mydot.dot")
#ROOT.RDF.SaveGraph(df)

#print(df_jme_lazy.GetValue())
#for b in branches:
#    print(b)
#    print(df_jme_lazy.GetValue())
    #print(df_jme_lazy[b].GetValue())

In [ ]:
'''
251853.0
250858.4424059987
317094.46073025465
9872941.734349836
286994.24495267123
CPU times: user 1min, sys: 4.71 s, total: 1min 4s
Wall time: 7min 55s

ALL
245838588.0
258514479.4809056
305701936.97344697
17764973396.952087
280127217.90940523
1821710343.4967484
70731458828.44531
nan
'''

In [ ]:
#chain[1]

In [ ]:
#import DistRDF
#DistRDF.__file__

In [ ]:
#df_saved = df_jme_lazy.GetValue()

In [ ]:
#df_snap = df_jme_lazy.GetValue()
#h = histos["FatJet_pt_nom"].GetValue()

In [ ]:
#c = ROOT.TCanvas()
#histos[branches[-1]].Draw()
#c.Draw()

In [ ]:
#c = ROOT.TCanvas()
#df_snap.Draw()
#c.Draw()

## Define preselection flow - Data

In [ ]:
#chain = ["root://xrootd-cms.infn.it//store/data/Run2017B/SingleElectron/NANOAOD/02Apr2020-v1/230000/1D3BEA1E-34E5-F04D-B470-5527915AC028.root"]
 #VBS_SSWW_LL_2017

if distributed == True:
    df_data = RDataFrame("Events", chain_data, npartitions=nPartitions, daskclient=client)
else:
    chain_data = ["root://xrootd-cms.infn.it//store/data/Run2017B/SingleElectron/NANOAOD/02Apr2020-v1/230000/1D3BEA1E-34E5-F04D-B470-5527915AC028.root"]
    #chain_data = ["1D3BEA1E-34E5-F04D-B470-5527915AC028.root"]
    df_data = RDataFrame("Events", chain_data).Range(10)

In [ ]:
%%time

df_year_data = df_data.DefinePerSample("Year","GetYear(rdfslot_, rdfsampleinfo_)")
df_sample_data = df_year_data.DefinePerSample("Sample", "GetSample(rdfslot_, rdfsampleinfo_)")

#### MET HLT Filter ####   #### working only for 2017 and 2018 (2016 requires a column that is not present in 2017 nd 2018, i.e. HLT_IsoTkMu24, 
#non c'è nei dati 2017: HLT_Ele32_WPTight_Gsf, lo setto a True dummy
df_METHLTFilter_data = df_sample_data.Define("METHLTFilter", "MET_HLT_Filter(Year, Flag_goodVertices, Flag_HBHENoiseFilter, Flag_HBHENoiseIsoFilter, Flag_EcalDeadCellTriggerPrimitiveFilter, Flag_BadPFMuonFilter, Flag_globalSuperTightHalo2016Filter, HLT_Ele27_WPTight_Gsf, true , HLT_IsoMu24, HLT_IsoMu27, HLT_Mu50, HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200)")\
                                     .Filter("METHLTFilter == true", "MET HLT Filter")

#### preselection #####
#df_preselection_data = df_METHLTFilter_data.Filter("PV_ndof> 4 && abs(PV_z) < 26 && hypot(PV_x, PV_y)<2", "Good vertex")\
#                                           .Define("HT_eventHT", "GetEventHT(Jet_pt, Jet_eta, Jet_phi, Jet_mass)")
df_preselection_data = df_METHLTFilter_data.Filter("PV_ndof> 4 && abs(PV_z) < 24 && hypot(PV_x, PV_y)<2", "Good vertex")\
                                           .Define("HT_eventHT", "GetEventHT(Jet_pt, Jet_eta, Jet_phi, Jet_mass)")\
                                           .Define("PassMinReq", "Pass_min_req(Muon_looseId, Muon_pt, Muon_pfRelIso04_all, Muon_eta, Electron_mvaFall17V2Iso_WPL, Electron_jetRelIso, Electron_pt, Electron_eta, Tau_idDeepTau2017v2p1VSjet, Tau_idDeepTau2017v2p1VSe, Tau_idDeepTau2017v2p1VSmu, Tau_pt, Tau_eta, Jet_pt, Jet_eta, Jet_puId)")\
                                           .Filter("PassMinReq == true", "Minimum requests")

#### lepSF #####
df_LepSF_data = df_preselection_data.Define("Electron_effSF", "ones(nElectron)")\
                                    .Define("Electron_effSF_errUp", "ones(nElectron)")\
                                    .Define("Electron_effSF_errDown", "ones(nElectron)")\
                                    .Define("Muon_effSF", "ones(nMuon)")\
                                    .Define("Muon_effSF_errUp", "ones(nMuon)")\
                                    .Define("Muon_effSF_errDown", "ones(nMuon)")


#### mht ####
df_mht_data = df_LepSF_data.Define("MHT_pt_phi", "MHT_pt_phi(Electron_pt, Electron_eta, Electron_phi, Electron_mass, Electron_miniPFRelIso_all, Muon_pt, Muon_eta, Muon_phi, Muon_mass, Muon_miniPFRelIso_all, Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_muonIdx1, Jet_muonIdx2, Jet_electronIdx1, Jet_electronIdx2, nJet)")\
                 .Define("MHT_pt", "MHT_pt_phi[0]").Define("MHT_phi", "MHT_pt_phi[1]")\


#### puWeight #####
df_puWeight_data = df_mht_data.Define("puWeight", "1.")\
                              .Define("puWeightUp", "1.")\
                              .Define("puWeightDown", "1.")


#### prefCorr ####
df_prefCorr_data = df_puWeight_data.Define("PrefireWeight", "1.")\
                                   .Define("PrefireWeightUp", "1.")\
                                   .Define("PrefireWeightDown", "1.")

#### ht ####
df_ht_data = df_prefCorr_data.Define("HT_pt", "htProducer(nJet, Jet_pt)")

#### btagSF ##### preselection part2
df_btagSF_data = df_ht_data.Define("Jet_btagSF_deepjet_M", "ones(nJet)")\
                           .Define("Jet_btagSF_deepjet_M_up", "ones(nJet)")\
                           .Define("Jet_btagSF_deepjet_M_down", "ones(nJet)")

#### muonScaleRes ####
df_muonScaleRes_data = df_btagSF_data.Define("muonCorrectedPTs", "muonScaleRes_data(Muon_pt, Muon_eta, Muon_phi, Muon_charge, Year)")\
                                     .Define("Muon_corrected_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 0)")\
                                     .Define("Muon_correctedUp_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 1)")\
                                     .Define("Muon_correctedDown_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 2)")

#### metCorrector and fatJetCorrector #### ####
df_jme_data = df_muonScaleRes_data.Define("jetVars", "myJetVarCalc.produce(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_rawFactor, Jet_area, Jet_jetId, fixedGridRhoFastjetAll, 0, {}, {}, {}, {})")\
                                  .Define("Jet_pt_nom", "jetVars.pt(0)")\
                                  .Define("Jet_mass_nom", "jetVars.mass(0)")\
                                  .Define("fatjetVars", "myFatJetVarCalc.produce(FatJet_pt, FatJet_eta, FatJet_phi, FatJet_mass, FatJet_rawFactor, FatJet_area, FatJet_msoftdrop, FatJet_subJetIdx1, FatJet_subJetIdx2, SubJet_pt, SubJet_eta, SubJet_phi, SubJet_mass, {}, fixedGridRhoFastjetAll, 0 , {}, {}, {}, {}, {}, {}, {}, {})")\
                                  .Define("FatJet_pt_nom", "fatjetVars.pt(0)")\
                                  .Define("FatJet_mass_nom", "fatjetVars.mass(0)")\
                                  .Define("FatJet_msoftdrop_nom", "fatjetVars.msoftdrop(0)")\
                                  .Define("metVars", "myType1METVarCalc.produce(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_rawFactor, Jet_area, Jet_muonSubtrFactor, Jet_neEmEF, Jet_chEmEF, Jet_jetId, fixedGridRhoFastjetAll, 0, {}, {}, {}, {}, RawMET_phi , RawMET_pt, MET_MetUnclustEnUpDeltaX, MET_MetUnclustEnUpDeltaY, CorrT1METJet_rawPt, CorrT1METJet_eta, CorrT1METJet_phi, CorrT1METJet_area, CorrT1METJet_muonSubtrFactor, {}, {})")\
                                  .Define("MET_T1_pt_nom", "metVars.pt(0)")\
                                  .Define("MET_T1_phi_nom", "metVars.phi(0)")\
                                  .Define("metsVars", "myType1SmearMETVarCalc.produce(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_rawFactor, Jet_area, Jet_muonSubtrFactor, Jet_neEmEF, Jet_chEmEF, Jet_jetId, fixedGridRhoFastjetAll, 0, {}, {}, {}, {}, RawMET_phi , RawMET_pt, MET_MetUnclustEnUpDeltaX, MET_MetUnclustEnUpDeltaY, CorrT1METJet_rawPt, CorrT1METJet_eta, CorrT1METJet_phi, CorrT1METJet_area, CorrT1METJet_muonSubtrFactor, {}, {})")\
                                  .Define("MET_T1Smear_pt_nom", "metsVars.pt(0)")\
                                  .Define("MET_T1Smear_phi_nom", "metsVars.phi(0)")
                        
for n,v in enumerate(jetvariations[1:]):
    df_jme_data = df_jme_data.Define("Jet_pt_{}".format(v), "jetVars.pt({})".format(1+n)).Define("Jet_mass_{}".format(v), "jetVars.mass({})".format(1+n))
for n,v in enumerate(fatjetvariations[1:]):
    df_jme_data = df_jme_data.Define("FatJet_pt_{}".format(v), "fatjetVars.pt({})".format(1+n)).Define("FatJet_mass_{}".format(v), "fatjetVars.mass({})".format(1+n)).Define("FatJet_msoftdrop_{}".format(v), "fatjetVars.msoftdrop({})".format(1+n))
for n,v in enumerate(mett1variations[1:]):
    df_jme_data = df_jme_data.Define("MET_T1_pt_{}".format(v), "metVars.pt({})".format(1+n)).Define("MET_T1_phi_{}".format(v), "metVars.phi({})".format(1+n))  
for n,v in enumerate(mett1smearvariations[1:]):
    df_jme_data = df_jme_data.Define("MET_T1Smear_pt_{}".format(v), "metsVars.pt({})".format(1+n)).Define("MET_T1Smear_phi_{}".format(v), "metsVars.phi({})".format(1+n))    


### add dummy columns ####
df_jme_data = df_jme_data.Define("Tau_genPartFlav", "ones(nTau)")\
                         .Define("Jet_partonFlavour", "ones(nJet)")

### book snapshot ###
opts = ROOT.RDF.RSnapshotOptions()
opts.fLazy = True

if distributed == True:
    #df_jme_data_lazy = df_jme_data.Snapshot("testTree", "root://stormgf3.pi.infn.it:1094//store/user/ttedesch/dummy_test/testFile_Data.root", branchlist, opts)
    df_jme_data_lazy = df_jme_data.Snapshot("testTree", "./preselectionUL_data.root", branchlist, opts)
    #df_jme_data_lazy = df_jme_data.Count()
else:
    df_jme_data_lazy = df_jme_data.Snapshot("testTree", "testFile_Data.root", branchlist, opts)

In [ ]:
df_snapshot_data = df_jme_data_lazy.GetValue()

In [ ]:
ERROR GENERATOR

## Launch the two computations concurrently and save results

In [ ]:
if distributed == True:
    RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs   
else:
    RunGraphs = ROOT.RDF.RunGraphs

proxies = [df_jme_lazy, df_jme_data_lazy]
RunGraphs(proxies)
dfs = [df_.GetValue() for df_ in proxies]

In [ ]:
for k,v in nevents_df.items():
    nevents_df[k] = v.GetValue()
for k,v in gen_weight_df.items():
    gen_weight_df[k] = v.GetValue()
with open('nEventsDict.json', 'w') as fp:
    json.dump(nevents_df, fp)
with open('genWeightDict.json', 'w') as fp:
    json.dump(gen_weight_df, fp)

## Debug

In [ ]:
#df_test =  ROOT.RDataFrame("testTree", "testFile_MC.root")
df_test = df_jme
#df_test.Display(["Electron_effSF", "Electron_effSF_errUp", "Electron_effSF_errDown", "Muon_effSF", "Muon_effSF_errUp", "Muon_effSF_errUp"], nRows = 5).Print()
#df_test.Display(["FatJet_pt_jesTotalup"], nRows = 10).Print()
#df_test.Display(["Muon_corrected_pt", "Muon_correctedUp_pt", "Muon_correctedDown_pt"], nRows = 5).Print()
#df_test.Display(["Jet_btagSF_deepjet_M_up","Jet_btagSF_deepjet_M_up", "Jet_btagSF_deepjet_M_down"]).Print()
#df_test.Display(["PrefireWeight" ,"PrefireWeightUp", "PrefireWeightDown"], nRows = 6).Print()
#df_test.Display(["Photon_pt", "Photon_eta", "Photon_jetIdx", "Photon_electronIdx"], nRows = 1).Print()
#df_test.Display(["Electron_pt", "Electron_eta", "Electron_jetIdx", "Electron_photonIdx"], nRows = 1).Print()
#df_test.Display(["Jet_pt", "Jet_eta"], nRows = 1).Print()
#df_test.Display(["MHT_pt" ,"MHT_phi"]).Print()
#df_test.Display(["HT_pt"]).Print()
#df_test.Display(["puWeight" ,"puWeightUp", "puWeightDown"]).Print()
#df_test.Display(["Jet_pt_nom", "Jet_pt_jerup", "Jet_pt_jerdown", "Jet_pt_jesTotalup", "Jet_pt_jesTotaldown"], nRows = 15).Print()
#df_test.Display(["Jet_mass_nom", "Jet_mass_jerup", "Jet_mass_jerdown", "Jet_mass_jesTotalup", "Jet_mass_jesTotaldown"], nRows = 15).Print()
#df_test.Display(["MET_T1_phi_nom", "MET_T1_phi_jerup", "MET_T1_phi_jerdown", "MET_T1_phi_jesTotalup", "MET_T1_phi_jesTotaldown"],  nRows = 15).Print()
#df_test.Display(["MET_T1Smear_pt_nom", "MET_T1Smear_pt_jerup", "MET_T1Smear_pt_jerdown"], nRows = 15).Print()
#df_test.Display(["MET_T1Smear_phi_nom", "MET_T1Smear_phi_jesTotalup", "MET_T1Smear_phi_jesTotaldown"], nRows = 15).Print()
#df_test.Display(["MET_T1Smear_pt_jesTotaldown", "MET_T1Smear_phi_jesTotalup", "MET_T1Smear_phi_jesTotaldown"]).Print()
#df_test.Display(["FatJet_pt_jesTotaldown", "FatJet_mass_jesTotalup", "FatJet_msoftdrop_jesTotaldown"]).Print()
df_test.Display(["FatJet_msoftdrop_nom", "FatJet_msoftdrop_jerup", "FatJet_msoftdrop_jerdown", "FatJet_msoftdrop_jesTotalup", "FatJet_msoftdrop_jesTotaldown"], nRows = 15).Print()
#df_test.Display(["Jet_mass_nom", "Jet_mass_jerup", "Jet_mass_jerdown", "Jet_mass_jesTotalup", "Jet_mass_jesTotaldown"], nRows = 15).Print()

In [ ]:
#df_classic = ROOT.RDataFrame("Events","https://ttedesch.web.cern.ch/ttedesch/testAF/VBS_SSWW_LL_SM_2017/tree_hadd_2.root")
df_classic = ROOT.RDataFrame("Events","https://ttedesch.web.cern.ch/ttedesch/VBS_SSWW_LL_SM_2017_tree_hadd_2_RR2017.root")
#df_test.Display(["ElectronSFs", "Electron_effSF", "MuonSFs", "Muon_effSF"], nRows = 20).Print()
#df_classic.Display(["Electron_effSF", "Electron_effSF_errUp", "Electron_effSF_errDown", "Muon_effSF", "Muon_effSF_errUp", "Muon_effSF_errUp"], nRows = 5).Print()
#df_classic.Display(["FatJet_pt_jesTotalUp"], nRows = 10).Print()
#df_classic.Display(["Muon_corrected_pt", "Muon_correctedUp_pt", "Muon_correctedDown_pt"], nRows = 5).Print()
#df_classic.Display(["Jet_btagSF_deepjet_M_up","Jet_btagSF_deepjet_M_up", "Jet_btagSF_deepjet_M_down"]).Print()
#df_classic.Display(["Jet_btagSF_deepjet_M_up","Jet_btagSF_deepjet_M_up", "Jet_btagSF_deepjet_M_down"]).Print()
#df_classic.Display(["PrefireWeight" ,"PrefireWeight_Up", "PrefireWeight_Down"], nRows = 6).Print()
#df_classic.Display(["Photon_pt", "Photon_eta", "Photon_jetIdx", "Photon_electronIdx"], nRows = 1).Print()
#df_classic.Display(["Electron_pt", "Electron_eta", "Electron_jetIdx", "Electron_photonIdx"], nRows = 1).Print()
#df_classic.Display(["Jet_pt", "Jet_eta"], nRows = 1).Print()
#df_classic.Display(["puWeight" ,"puWeightUp", "puWeightDown"]).Print()
#df_test.Display(["puWeight" ,"puWeightUp", "puWeightDown"]).Print()
#df_classic.Display(["MHT_pt" ,"MHT_phi"]).Print()
#df_classic.Display(["HT_pt"]).Print()
#df_classic.Display(["Jet_pt_jesTotalUp", "Jet_pt_jesTotalDown"]).Print()
#df_classic.Display(["Jet_mass_nom", "Jet_mass_jerUp", "Jet_mass_jerDown", "Jet_mass_jesTotalUp", "Jet_mass_jesTotalDown"], nRows = 20).Print()
#df_classic.Display(["MET_T1_phi", "MET_T1_phi_jerUp", "MET_T1_phi_jerDown", "MET_T1_phi_jesTotalUp", "MET_T1_phi_jesTotalDown"],  nRows = 15).Print()
#df_classic.Display(["MET_T1Smear_pt", "MET_T1Smear_pt_jerUp", "MET_T1Smear_pt_jerDown",], nRows = 15).Print()
#df_classic.Display(["MET_T1Smear_phi", "MET_T1Smear_phi_jesTotalUp", "MET_T1Smear_phi_jesTotalDown"], nRows = 15).Print()
#df_classic.Display(["MET_T1_pt_jesTotalDown", "MET_T1_phi_jesTotalUp", "MET_T1_phi_jesTotalDown"]).Print()
#df_classic.Display(["MET_T1Smear_pt_jesTotalDown", "MET_T1Smear_phi_jesTotalUp", "MET_T1Smear_phi_jesTotalDown"]).Print()
df_classic.Display(["FatJet_msoftdrop_nom", "FatJet_msoftdrop_jerUp", "FatJet_msoftdrop_jerDown", "FatJet_msoftdrop_jesTotalUp", "FatJet_msoftdrop_jesTotalDown"], nRows = 15).Print()
#df_classic.Display(["FatJet_pt_jesTotalDown", "FatJet_mass_jesTotalUp", "FatJet_msoftdrop_jesTotalDown"]).Print()

In [ ]:
'''
df_test =  RDataFrame("testTree", "testFile.root")
#df_test.Display(["ElectronSFs", "Electron_effSF", "MuonSFs", "Muon_effSF"], nRows = 20).Print()
df_test.Display(["Jet_pt", "Jet_eta", "Jet_hadronFlavour", "Jet_btagDeepFlavB"]).Print()
'''

In [ ]:
#print(df_classic.GetColumnNames())